# Fetch Open Datasets of PV locations

Many of these datasets are located in [Zenodo](https://zenodo.org/), a general-purpose open-access repository developed under the European OpenAIRE program and operated by CERN. Others are hosted in figshare, a web-based platform for sharing research data and other types of content. The rest are hosted in GitHub repositories or other open-access platforms.The datasets are available in various formats, including CSV, GeoJSON, and shapefiles, and raster masks. We'll be using open-source Python libraries to download and process them into properly georeferenced geoparquet files that will serve as a base for our [OLAP](https://www.datacamp.com/blog/oltp-vs-olap) database files that we'll manage with [dbt-core+duckdb](https://motherduck.com/blog/motherduck-duckdb-dbt/) (see these short videos on [this stack](https://www.youtube.com/watch?v=asxGh2TrNyI) and background on [MotherDuck](https://www.youtube.com/watch?v=OuCY7_DzCTA) for local AND cloud geospatial data processing with the same tools and codebase).

Here we list the dataset titles alongside their first author, DOI links, and their number of labels:
- **"Distributed solar photovoltaic array location and extent dataset for remote sensing object identification"** - K. Bradbury, 2016 | [paper DOI](https://doi.org/10.1038/sdata.2016.106) | [dataset DOI](https://doi.org/10.6084/m9.figshare.3385780.v4) | polygon annotations for 19,433 PV modules in 4 cities in California, USA
- "A solar panel dataset of very high resolution satellite imagery to support the Sustainable Development Goals" - C. Clark et al, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02539-8) | [dataset DOI](https://doi.org/10.6084/m9.figshare.22081091.v3) | 2,542 object labels (per spatial resolution)
- "A harmonised, high-coverage, open dataset of solar photovoltaic installations in the UK" - D. Stowell et al, 2020 | [paper DOI](https://doi.org/10.1038/s41597-020-00739-0) | [dataset DOI](https://zenodo.org/records/4059881) | 265,418 data points (over 255,000 are stand-alone installations, 1067 solar farms, and rest are subcomponents within solar farms)
- "Georectified polygon database of ground-mounted large-scale solar photovoltaic sites in the United States" - K. Sydny, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-02644-8) | [dataset DOI](https://www.sciencebase.gov/catalog/item/6671c479d34e84915adb7536) | 4186 data points (Note: these correspond to PV _facilities_ rather than individual panel arrays or objects and need filtering of duplicates with other datasets and further processing to extract the PV arrays in the facility)
- "Vectorized solar photovoltaic installation dataset across China in 2015 and 2020" - J. Liu et al, 2024 | [paper DOI](https://doi.org/10.1038/s41597-024-04356-z) | [dataset link](https://github.com/qingfengxitu/ChinaPV) | 3,356 PV labels (inspect quality!)
- "Multi-resolution dataset for photovoltaic panel segmentation from satellite and aerial imagery" - H. Jiang, 2021 | [paper DOI](https://doi.org/10.5194/essd-13-5389-2021) | [dataset DOI](https://doi.org/10.5281/zenodo.5171712) | 3,716 samples of PV data points
- "A crowdsourced dataset of aerial images with annotated solar photovoltaic arrays and installation metadata" - G. Kasmi, 2023 | [paper DOI](https://doi.org/10.1038/s41597-023-01951-4) | [dataset DOI](https://doi.org/10.5281/zenodo.6865878) | > 28K points of PV installations; 13K+ segmentation masks for PV arrays; metadata for 8K+ installations
- **"An Artificial Intelligence Dataset for Solar Energy Locations in India"** - A. Ortiz, 2022 | [paper DOI](https://doi.org/10.1038/s41597-022-01499-9) | [dataset link 1](https://researchlabwuopendata.blob.core.windows.net/solar-farms/solar_farms_india_2021.geojson) or [dataset link 2](https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson) | 117 geo-referenced points of solar installations across India
- "GloSoFarID: Global multispectral dataset for Solar Farm IDentification in satellite imagery" - Z. Yang, 2024** | [paper DOI](https://doi.org/10.48550/arXiv.2404.05180) | [dataset DOI](https://github.com/yzyly1992/GloSoFarID/tree/main/data_coordinates) | 6,793 PV samples across 3 years (double counting of samples)
- **"A global inventory of photovoltaic solar energy generating units" - L. Kruitwagen et al, 2021** | [paper DOI](https://doi.org/10.1038/s41586-021-03957-7) | [dataset DOI](https://doi.org/10.5281/zenodo.5005867) | 50,426 for training, cross-validation, and testing; 68,661 predicted polygon labels 
- **"Harmonised global datasets of wind and solar farm locations and power" - S. Dunnett et al, 2020** | [paper DOI](https://doi.org/10.1038/s41597-020-0469-8) | [dataset DOI](https://doi.org/10.6084/m9.figshare.11310269.v6) | 35272 PV installations

In [ ]:
from IPython.display import display
from IPython.display import clear_output
from tqdm import tqdm
import ipywidgets as widgets
from ipywidgets import Layout
from dotenv import load_dotenv

import numpy as np
import xarray as xr
from branca.colormap import linear
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
# import matplotlib.pyplot as plt

import geopandas as gpd
import rasterio
import shapely
import pygeohash
import folium
import lonboard
import pydeck as pdk
# import openeo 
# import pystac_client

# import easystac
# import cubo

import duckdb as dd 
import datahugger
import sciencebasepy
from seedir import seedir

# python libraries
import os
import json
import requests
import urllib.parse
from pathlib import Path
import subprocess
import tempfile
import shutil
import pprint as pp
import time
import re
from zipfile import ZipFile
import random

# Dataset Metadata and Preparation

We will use datahugger, sciencebasepy, and osf-client to manage official dataset records published in open access scientific data repositories.
We also implement some ad-hoc functions to download some data assets hosted in GitHub repositories. 
We will also use geopandas, rasterio, and pyproj to process the datasets into georeferenced geoparquet files. 

In [ ]:
# create dict of metadata for datasets
# this will be used for interactive widget and managing downloads

# for maxar dataset
# Catalogue ID 1040050029DC8C00; use to find geospatial extent coords
# The geocoordinates for each solar panel object may be determined using the native resolution labels (found in the labels_native directory). 
# The center and width values for each object, along with the relative location information provided by the naming convention for each label, 
# may be used to determine the pixel coordinates for each object in the full, corresponding native resolution tile. 
# The pixel coordinates may be translated to geocoordinates using the EPSG:32633 coordinate system and the following geotransform for each tile:

# Tile 1: (307670.04, 0.31, 0.0, 5434427.100000001, 0.0, -0.31)
# Tile 2: (312749.07999999996, 0.31, 0.0, 5403952.860000001, 0.0, -0.31)
# Tile 3: (312749.07999999996, 0.31, 0.0, 5363320.540000001, 0.0, -0.31)
# see here on gdal format geotransform: https://gdal.org/en/stable/tutorials/geotransforms_tut.html

# look into adding dataset crs or projection to metadata dict
# note that most of these details are hardcoded and difficult to parse ahead of time
# load environment variables
load_dotenv()
DATASET_DIR = Path(os.getenv('DATA_PATH'))
dataset_metadata = {
    'deu_maxar_vhr_2023': {
        'doi': '10.6084/m9.figshare.22081091.v3',
        'repo': 'figshare',
        'compression': 'zip',
        'label_fmt': 'yolo_fmt_txt',
        'has_imgs': False,
        'label_count': 2542 # solar panel objects (ie not individual panels)
    },
    'uk_crowdsourced_pv_2020': {
        'doi': '10.5281/zenodo.4059881',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'geojson',
        'geom_type': {'features': ['Point', 'Polygon', 'MultiPolygon']},
        'crs': None, # default to WGS84 when processing
        'has_imgs': False,
        'label_count': 265418
    },
    # note for report later: Maxar Technologies (MT) was primarily used to determine the extent of solar arrays
    'usa_eia_large_scale_pv_2023': {
        'doi': '10.5281/zenodo.8038684',
        'repo': 'sciencebase',
        'compression': 'zip',
        'label_fmt': 'shp',
        'has_imgs': False,
        'label_count': 4186
    },
    'chn_med_res_pv_2024': {
        # using github files since zenodo shapefiles fail to load in QGIS
        'doi': 'https://github.com/qingfengxitu/ChinaPV/tree/main',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'shp',
        'has_imgs': False,
        'label_count': 3356
    },
    'usa_cali_usgs_pv_2016': {
        'doi': '10.6084/m9.figshare.3385780.v4',
        'repo': 'figshare',
        'compression': None,
        'label_fmt': 'geojson',
        'crs': 'NAD83',
        'geom_type': {'features': 'Polygon'},
        'has_imgs': False,
        'label_count': 19433
    },
    'chn_jiangsu_vhr_pv_2021': {
        'doi': '10.5281/zenodo.5171712',
        'repo': 'zenodo',
        'compression': 'zip',
        # look into geotransform details for processing these labels
        'label_fmt': 'pixel_mask',
        'has_imgs': True,
        'label_count': 3716
    },
    'ind_pv_solar_farms_2022': {
        'doi': 'https://raw.githubusercontent.com/microsoft/solar-farms-mapping/refs/heads/main/data/solar_farms_india_2021_merged_simplified.geojson',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'geojson',
        'geom_type': {'features': 'MultiPolygon'}, 
        'crs': 'WGS84',
        'has_imgs': False,
        'label_count': 117
    },
    'fra_west_eur_pv_installations_2023': {
        'doi': '10.5281/zenodo.6865878',
        'repo': 'zenodo',
        'compression': 'zip',
        'label_fmt': 'json',
        'geom_type': {'Polygon': ['Point']},
        'crs': None, 
        'has_imgs': True, 
        'label_count': (13303, 7686)
    },
    'global_pv_inventory_sent2_spot_2021': {
        'doi': '10.5281/zenodo.5005867',
        'repo': 'zenodo',
        'compression': None,
        'label_fmt': 'geojson',
        'geom_type': ['Polygon'],
        'crs': 'WGS84',
        'has_imgs': False,
        'label_count': (50426, 68661) # (training+cv+test, human-verified predictions)
    },
    'global_pv_inventory_sent2_2024': {
        'doi': 'https://github.com/yzyly1992/GloSoFarID/tree/main/data_coordinates',
        'repo': 'github',
        'compression': None,
        'label_fmt': 'json',
        'crs': None, # default to WGS84 when processing
        'geom_type': ['Point'], # normal json with no geometry attribute
        'has_imgs': True, 
        'label_count': 6793
    },
    'global_harmonized_large_solar_farms_2020': {
        'doi': '10.6084/m9.figshare.11310269.v6',
        'repo': 'figshare',
        'compression': 'zip', # entire dataset is zipped
        'label_fmt': 'gpkg',
        'geom_type': ['Polygon'], # only in geopackage; all other formats are coords of centroid points
        'crs': 'WGS84', # also offers file in Eckert IV projection
        'has_imgs': False,
        'label_count': 35272

    }

}

dataset_choices = [
    'global_harmonized_large_solar_farms_2020',
    # 'global_pv_inventory_sent2_2024',
    'global_pv_inventory_sent2_spot_2021',
    # 'fra_west_eur_pv_installations_2023',
    'ind_pv_solar_farms_2022',
    'usa_cali_usgs_pv_2016',
    # 'chn_med_res_pv_2024',
    # 'usa_eia_large_scale_pv_2023',
    # 'uk_crowdsourced_pv_2020',
    # 'deu_maxar_vhr_2023'   
]

In [ ]:
# Initialize a list to store selected datasets
# mostly gen by github copilot with Claude 3.7 model
selected_datasets = dataset_choices.copy()

def format_dataset_info(dataset):
    """Create a formatted HTML table for dataset metadata"""
    metadata = dataset_metadata[dataset]
    
    # Create table with metadata
    html = f"""
    <style>
    .dataset-table {{
        border-collapse: collapse;
        width: 30%;
        margin: 20px auto;
        font-family: Arial, sans-serif;
    }}
    .dataset-table th, .dataset-table td {{
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
    }}
    .dataset-table th {{
        background-color: #f2f2f2;
        font-weight: bold;
    }}
    </style>
    <table class="dataset-table">
        <tr><th>Metadata</th><th>Value</th></tr>
        <tr><td>DOI/URL</td><td>{metadata['doi']}</td></tr>
        <tr><td>Repository</td><td>{metadata['repo']}</td></tr>
        <tr><td>Compression</td><td>{metadata['compression'] or 'None'}</td></tr>
        <tr><td>Label Format</td><td>{metadata['label_fmt']}</td></tr>
        <tr><td>Has Images</td><td>{metadata['has_imgs']}</td></tr>
        <tr><td>Label Count</td><td>{metadata.get('label_count', 'Unknown')}</td></tr>
    </table>
    """
    return html

# Create an accordion to display selected datasets with centered layout
dataset_accordion = widgets.Accordion(
    children=[widgets.HTML(format_dataset_info(ds)) for ds in selected_datasets],
    layout=Layout(width='50%', margin='0 auto')
)
for i, ds in enumerate(selected_datasets):
    dataset_accordion.set_title(i, ds)

# Define a function to add or remove datasets
def manage_datasets(action, dataset=None):
    global selected_datasets, dataset_accordion
    
    if action == 'add' and dataset and dataset not in selected_datasets:
        selected_datasets.append(dataset)
    elif action == 'remove' and dataset and dataset in selected_datasets:
        selected_datasets.remove(dataset)
    
    # Update the accordion with current selections
    dataset_accordion.children = [widgets.HTML(format_dataset_info(ds)) for ds in selected_datasets]
    for i, ds in enumerate(selected_datasets):
        dataset_accordion.set_title(i, ds)
    
    f"Currently selected datasets: {len(selected_datasets)}"

# Create dropdown for available datasets
dataset_dropdown = widgets.Dropdown(
    options=list(dataset_metadata.keys()),
    description='Dataset:',
    disabled=False,
    layout=Layout(width='70%', margin='20 20 auto 20 20')
)

# Create buttons for actions
add_button = widgets.Button(description="Add Dataset", button_style='success')
remove_button = widgets.Button(description="Remove Dataset", button_style='danger')

# Define button click handlers
def on_add_clicked(b):
    manage_datasets('add', dataset_dropdown.value)

def on_remove_clicked(b):
    manage_datasets('remove', dataset_dropdown.value)

# Link buttons to handlers
add_button.on_click(on_add_clicked)
remove_button.on_click(on_remove_clicked)

## Dataset Selection Interface
#### Use the dropdown and buttons below to customize which solar panel datasets will be fetched and processed.
- Select a dataset from the dropdown:
    - Click "Add Dataset" to include it in processing
    - Click "Remove Dataset" to exclude it
- View metadata table in the selected dataset's dropdown

In [ ]:
# Display the widgets
display(widgets.HBox([dataset_dropdown, add_button, remove_button]))
display(dataset_accordion)

# Fetching and Organizing datasets for later-preprocessing

We will use [datahugger](https://j535d165.github.io/datahugger/) to fetch datasets hosted in Zenodo, figshare, and GitHub. 

We will sciencebase for the dataset hosted in the USGS ScienceBase Catalog.
We will pre-process and convert datasets into geojson, if not already formatted, and manage these using [geopandas](https://geopandas.org/). These will be further processed into geoparquet files for use in duckdb tables used to manage and later consolidate the datasets with dbt.  
- The datasets will be stored in the `data/` directory
    - the geoparquet files will be stored in the `data/geoparquet/` directory

#### Processing

In [ ]:
# move to utility functions later
# def fetch_github_repo_files(dataset_name, 


# use the metadata to fetch the dataset files using datahugger
def fetch_dataset_files(dataset_name, max_mb=100, force=False):
    metadata = dataset_metadata[dataset_name]
    doi = metadata['doi']
    repo = metadata['repo']
    compression = metadata['compression']
    label_fmt = metadata['label_fmt']
    # convert to bytes
    max_dl = max_mb * 1024 * 1024
    dataset_dir = os.path.join(os.getenv('DATA_PATH'), 'raw', 'labels', dataset_name)
    geofile_regex = r'^(.*\.(geojson|json|shp|zip|csv|gpkg))$'
    dst = os.path.join(os.getcwd(), dataset_dir)
    dst_p = Path(dst)

    # if re-fetching, remove existing files to avoid duplicates
    if force and dst_p.exists():
        shutil.rmtree(dst_p)
        print(f"Removed existing dataset directory: {os.path.relpath(dst_p)}")

    # prettyprint metadata and dst info
    # pp.pprint(metadata)
    # print(f"Destination: {dataset_dir}")
    # print(f"Max download size: {max_mb} MB")
    # print(f"Force Download: {force}")

    dataset_tree = {}

    # TODO: move different repo handling to separate functions

    # use datahugger to fetch files from most repos
    if repo in ['figshare', 'zenodo']:

        # handle unzipping manually below so we get progress bar for download
        ds_tree = datahugger.get(doi, dst, max_file_size=max_dl, force_download=force, unzip=False)
        # compare files to be fetched (after filtering on max file size) with existing files  
        files_to_fetch = [f['name'] for f in ds_tree.dataset.files if f['size'] <= max_dl]
        ds_files = [os.path.join(root, fname) for root, dirs, files in os.walk(dst_p) for fname in files if re.match(geofile_regex, fname)]
        # flag for avoiding extracting zip when already extracted
        # is_unzipped = all(f in ds_files for f in files_to_fetch) and len(ds_files) > 1
        # TODO: handle .zip files that consist of a redundant copy of the entire dataset
        if metadata['compression'] == 'zip' and any(f.endswith('.zip') for f in ds_files):
            print(f"Dataset metadata for {dataset_name} indicates handling of one or more downloaded zip files.")
            # check if the zip file was fetched and directly extract if it's the only file in the dataset
            extracted_files = []
            if len(ds_files) <= 2 and ds_files[0].endswith('.zip'):
                zip_file = dst_p / ds_files[0]
                # print(f"Found single zip file for dataset: {zip_file}")
                # extract the zip file and delete it 
                with ZipFile(zip_file, 'r') as zip_ref:
                    extracted_files = zip_ref.namelist()
                    zip_ref.extractall(dst)
                
                # remove the zip file
                # try:
                #     os.remove(zip_file)
                #     print(f"Removed {os.path.relpath(zip_file)} after extraction")
                # except Exception as e:
                #     print(f"Error removing {zip_file}: {e}")
                # check if zip file consisted of a single dir and move contents up one level
                top_level_dir = dst_p / extracted_files[0]
                if top_level_dir.is_dir():
                    # move only first level dirs and files to our dataset dir
                    for item in top_level_dir.iterdir():
                        if item.name.endswith('.zip'):
                            continue
                        # don't copy if already exists and is non-empty
                        # TODO: add non-empty check
                        elif os.path.exists(dst_p / item.name):
                            print(f"Skipping {item} as it already exists in {os.path.relpath(dst)}")
                            continue
                        elif item.parent == top_level_dir and re.match(geofile_regex, item.name):
                            print(f"Moving {item} to {os.path.relpath(dst)}")
                            shutil.move(item, dst)
                    # remove the top level dir
                    shutil.rmtree(top_level_dir)

                ds_files = [os.path.join(root, fname) for root, dirs, files in os.walk(dst_p) for fname in files if re.match(geofile_regex, fname)]
                print(f"Moved items from {os.path.relpath(top_level_dir)} to:\n{os.path.relpath(dst_p)}")
                print(f"After extraction and moving, we have {len(ds_files)} files in {os.path.relpath(dst)}:\n{ds_files}")

            elif len(ds_files) > 2:
                # multiple files in addition to the zip file; handle on case by case basis
                print(f"Multiple files found in {dst_p}:\n{os.listdir(dst_p)}")
        # no further processing needed; get file list directly from datahugger
        else: 
            ds_files = [os.path.join(root, fname) for root, dirs, files in os.walk(dst_p) for fname in files if re.match(geofile_regex, fname)]


        dataset_tree = {
            'dataset': dataset_name,
            'output_dir': ds_tree.output_folder,
            'files': ds_files,
            'fs_tree': seedir(dst_p, depthlimit=5, printout=False, regex=True, include_files=geofile_regex)
        }

    elif repo == 'github':
        # Handle GitHub repositories using git partial cloning of repo 
        
        # Create destination directory if it doesn't exist
        os.makedirs(dst, exist_ok=True)
        # Parse the GitHub URL
        # [user, repo, tree, branch, rest of path]
        parts = doi.replace('https://github.com/', '').split('/')
        repo_path = f"{parts[0]}/{parts[1]}"
        
        # Extract branch and path
        branch = 'main'  # Default branch
        path = ''
        
        # check if local path exists and contains expected files
        if os.path.exists(dst) and any(os.path.splitext(fname)[1] in ['.geojson', '.json', '.shp', '.zip'] for fname in os.listdir(dst)) and not force:  
            print(f"Destination path for {dataset_name}'s repo already exists and contains expected files.")
            # print in bold
            print(f"\033[1mSkipping Download!\033[0m")
            # fetch dataset dir info from Pathlib and tree from seedir 
            tree = seedir(dst_p, depthlimit=5, printout=False, regex=True, include_files=geofile_regex)
            # get list of files in Path object that satisfy regex
            ds_files = [os.path.join(root, fname) for root, dirs, files in os.walk(dst_p) for fname in files if re.match(geofile_regex, fname)]
            dataset_tree = {
                'dataset': dataset_name,
                'output_dir': dst,
                'files': ds_files,
                'fs_tree': tree
            }

        # Check if it's a folder/repository or a single file
        elif '/blob/' not in doi and 'raw.githubusercontent.com' not in doi:
            try:
                if 'tree' in parts:
                    tree_index = parts.index('tree')
                    branch = parts[tree_index + 1]
                    path = '/'.join(parts[tree_index + 2:]) if len(parts) > tree_index + 2 else ''
                
                # Create a temporary directory for the sparse checkout
                with tempfile.TemporaryDirectory() as temp_dir:
                    # Initialize the git repository and set up sparse checkout
                    commands = [f"git clone --filter=blob:limit={max_mb}m --depth 1 https://github.com/{repo_path}.git {dataset_name}"]
                    # print(f"Running commands: {commands}")
                    # Execute git commands
                    for cmd in commands:
                        
                        process = subprocess.run(cmd, shell=True, cwd=temp_dir, 
                                               capture_output=True, text=True)
                        # show command output (debug)
                        print(f"Command stdout: {process.stdout}")
                        if process.returncode != 0:
                            raise Exception(f"Git command failed: {cmd}\n{process.stderr}")
                    
                    # Copy only the files in the dir specified in DOI/URL
                    repo_ds_dir = os.path.join(temp_dir, dataset_name, path) if path else os.path.join(temp_dir, dataset_name)
                    files_list = []
                    #
                    for root, _, files in os.walk(repo_ds_dir):
                        for file in files:
                            if file.startswith('.git'):
                                continue
                            src_file = os.path.join(root, file)
                            # Create relative path
                            rel_path = os.path.relpath(src_file, repo_ds_dir)
                            dst_file = os.path.join(dst, rel_path)
                            
                            # Create destination directory if needed
                            os.makedirs(os.path.dirname(dst_file), exist_ok=True)
                            
                            # Copy the file
                            shutil.copy2(src_file, dst_file)
                            files_list.append(dst_file)
                            print(f"Copied {rel_path} to ./{dataset_dir}/{rel_path}")

                dataset_tree = {
                    'dataset': dataset_name,
                    'output_dir': dst,
                    'files': files_list,
                    'fs_tree': seedir(dst_p, depthlimit=5, printout=False, regex=True, include_files=geofile_regex)
                }
                
            except Exception as e:
                print(f"Error performing git clone: {e}")
                return None
        else:
            # It's a single file (raw URL or blob URL)
            try:
                # Convert blob URL to raw URL if needed
                if '/blob/' in doi:
                    raw_url = doi.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
                else:
                    raw_url = doi
                
                # Extract filename from URL
                filename = os.path.basename(urllib.parse.urlparse(raw_url).path)
                local_file_path = os.path.join(dst, filename)
                
                # Download the file
                response = requests.get(raw_url, stream=True)
                response.raise_for_status()
                
                # Check file size
                file_size = int(response.headers.get('content-length', 0))
                if file_size > max_dl:
                    print(f"File size ({file_size} bytes) exceeds maximum allowed size ({max_dl * 1024 * 1024} MB)")
                    return None
                
                with open(local_file_path, 'wb') as f:
                    for chunk in tqdm(response.iter_content(chunk_size=8192), desc=f"Downloading {filename}", unit='KB'):
                        f.write(chunk)
                print(f"Downloaded {filename} to {os.path.relpath(local_file_path)}")
                dataset_tree = {
                    'dataset': dataset_name,
                    'output_dir': dst,
                    'files': [local_file_path],
                    'fs_tree': seedir(dst_p, depthlimit=5, printout=False, regex=True, include_files=geofile_regex)
                }
                
            except Exception as e:
                print(f"Error downloading GitHub file: {e}")

    elif repo == 'sciencebase':
        # Initialize ScienceBase client
        # sb = sciencebasepy.SbSession()
        
        # # Extract the item ID from the DOI or URL
        # # DOIs like 10.5281/zenodo.8038684 or URLs with item ID
        # item_id = doi.split('/')[-1] if '/' in doi else doi
        
        # try:
        #     # Get item details
        #     item = sb.get_item(item_id)
            
        #     # Create destination directory
        #     os.makedirs(dst, exist_ok=True)
            
        #     # Download all files associated with the item
        #     downloaded_files = []
            
        #     # Get item files
        #     files = sb.get_item_file_info(item_id)
            
        #     for file_info in files:
        #         file_name = file_info['name']
        #         file_url = file_info['url']
                
        #         # Check file size if available
        #         if 'size' in file_info and file_info['size'] > max_dl:
        #             print(f"Skipping file {file_name} as it exceeds the maximum download size")
        #             continue
                
        #         # Download the file
        #         local_file_path = os.path.join(dst, file_name)
        #         sb.download_file(file_url, local_file_path)
                
        #         downloaded_files.append(local_file_path)
        #         print(f"Downloaded {file_name} to {local_file_path}")
        print("Not Implemented yet")
        return None

    print(f"Fetched {len(dataset_tree['files'])} dataset files for {dataset_name} in {os.path.relpath(dataset_tree['output_dir'])}:")
    print(dataset_tree['fs_tree'])

    return dataset_tree

In [ ]:
# iterate through the selected datasets and fetch files
# iterate through the selected datasets and fetch files
ds_trees = {}
max_mb = int(os.getenv('MAX_LABEL_MB', 100))
print(f"Fetching {len(selected_datasets)} datasets with files of max size {max_mb} MB")

# Create widgets for controlling the fetching process
fetch_output = widgets.Output(
    layout=widgets.Layout(
        width='80%', 
        border='1px solid #ddd', 
        padding='10px',
        overflow='auto'
    )
)
# Apply direct CSS styling for text wrapping (Note: unvalidated)
display(widgets.HTML("""
<style>
.jupyter-widgets-output-area pre {
    white-space: pre-wrap !important;       /* CSS3 */
    word-wrap: break-word !important;        /* Internet Explorer 5.5+ */
    overflow-wrap: break-word !important;
    max-width: 100%;
}
</style>
"""))
control_panel = widgets.VBox(layout=widgets.Layout(width='20%', padding='10px', overflow='auto', word_wrap='break-word'))
fetch_button = widgets.Button(description="Fetch Next Dataset", button_style="primary")
progress_label = widgets.HTML("Waiting to start...")
dataset_index = 0

# Function to fetch the next dataset
def fetch_next_dataset(button=None):
    global dataset_index
    global dataset_metadata
    
    if dataset_index >= len(selected_datasets):
        with fetch_output:
            print("All datasets have been fetched!")
            progress_label.value = f"<b>Completed:</b> {dataset_index}/{len(selected_datasets)} datasets"
        fetch_button.disabled = True
        return
    
    dataset = selected_datasets[dataset_index]
    progress_label.value = f"<b>Fetching:</b> {dataset_index+1}/{len(selected_datasets)}<br><b>Current:</b> {dataset}"
    
    with fetch_output:
        clear_output(wait=True)
        print(f"Fetching dataset files for {dataset} using DOI/URL:\n {dataset_metadata[dataset]['doi']}")
        ds_tree = fetch_dataset_files(dataset, max_mb=max_mb, force=force_download_checkbox.value)
        
        if ds_tree:
            ds_trees[dataset] = ds_tree
            # update metadata dict with local filesystem info
            dataset_metadata[dataset]['output_dir'] = ds_tree['output_dir']
            dataset_metadata[dataset]['files'] = ds_tree['files']
            dataset_metadata[dataset]['tree'] = ds_tree['fs_tree']
            # print the dataset file tree
        else:
            print(f"Failed to fetch dataset {dataset}")
    
    dataset_index += 1
    progress_label.value = f"<b>Completed:</b> {dataset_index}/{len(selected_datasets)}<br><b>Next:</b> {selected_datasets[dataset_index] if dataset_index < len(selected_datasets) else 'Done'}"

# Add a checkbox for force download option
force_download_checkbox = widgets.Checkbox(
    value=False,
    description='Force Download',
    tooltip='If checked, download will be forced even if files exist locally',
    layout=widgets.Layout(width='auto')
)

# Configure the button callback
fetch_button.on_click(fetch_next_dataset)

# Create the control panel
dataset_progress = widgets.HTML(f"Datasets selected: {len(selected_datasets)}")
fetch_status = widgets.HTML(
    f"Status: Ready to begin",
    layout=widgets.Layout(margin="10px 0")
)

# Create the control panel with left alignment
control_panel.children = [
    widgets.HTML("<h3 style='align:left;'>Fetch Control</h3>"), 
    dataset_progress,
    force_download_checkbox,
    widgets.HTML("<hr style='margin:10px 0'>"),
    progress_label,
    fetch_button
]

# Add custom CSS to ensure alignment
display(widgets.HTML("""
<style>
.widget-html {
    text-align: left !important;
}
.widget-checkbox {
    justify-content: flex-start !important;
}
.widget-button {
    width: 100% !important;
}
</style>
"""))

#### Fetching selected datasets and visualizing metadata and file structure

Use the simple UI rendered below to click the "Next Dataset" button to initiate the fetching of the selected datasets above. 
The datasets will avoid redownloading existing files, but the user can force a re-download by checking the "Force Re-download" checkbox. 

We will store the datasets in the `DATA_PATH` variable configured in the repo's `.env` file in the `raw/` subdirectory (to denote data fetched and saved as-is or with minimal processing).



In [ ]:
# check if files and fs_tree are empty for selected datasets (ie user did not use fetch UI above or there was an error with one of the fetches)
processed_ds_keys = ['output_dir', 'files', 'tree']
missing_ds_fetches = [ds for ds in selected_datasets if any(key not in dataset_metadata[ds] for key in processed_ds_keys)]
if missing_ds_fetches:
    print(f"Warning: The following datasets were not fetched or processed correctly:\n{missing_ds_fetches}")
    print(f"We will attempt to fetch these in a loop in the next cell, but if you did not skip the fetch step, please check the stdout above for errors.")
else:
    print(f"All {len(selected_datasets)} datasets have been fetched and processed correctly (🤞).")

if missing_ds_fetches:
    # loop through the datasets that were not fetched and try to fetch them again
    for dataset in missing_ds_fetches:
        print(f"Attempting to fetch {dataset} again...")
        ds_tree = fetch_dataset_files(dataset, max_mb=max_mb, force=False)
        if ds_tree:
            ds_trees[dataset] = ds_tree
            # update metadata dict with local filesystem info
            dataset_metadata[dataset]['output_dir'] = ds_tree['output_dir']
            dataset_metadata[dataset]['files'] = ds_tree['files']
            dataset_metadata[dataset]['tree'] = ds_tree['fs_tree']
        else:
            print(f"Failed to fetch dataset {dataset} again; Error is likely in the fetch function above.")

In [ ]:
# print(datahugger.get.__doc__)

#### Global inventory of solar PV units (Kruitwagen et al, 2021)

From Zenodo:
```
Repository contents:

trn_tiles.geojson: 18,570 rectangular areas-of-interest used for sampling training patch data.

trn_polygons.geojson: 36,882 polygons obtained from OSM in 2017 used to label training patches.

cv_tiles.geojson: 560 rectangular areas-of-interest used for sampling cross-validation data seeded from WRI GPPDB

cv_polygons.geojson: 6,281 polygons corresponding to all PV solar generating units present in cv_tiles.geojson at the end of 2018.

test_tiles.geojson: 122 rectangular regions-of-interest used for building the test set.

test_polygons.geojson: 7,263 polygons corresponding to all utility-scale (>10kW) solar generating units present in test_tiles.geojson at the end of 2018.

predicted_polygons.geojson: 68,661 polygons corresponding to predicted polygons in global deployment, capturing the status of deployed photovoltaic solar energy generating capacity at the end of 2018.
``` 

**Regarding predicted_polygons.geojson**: "The final dataset includes 68,661 detections in 131 countries with a mean detection
area of approximately 70,000m2. Country-level aggregates are shown in Supplementary
Table 10. Any false positives remaining in the dataset are a product of human error. Final precision statistics in Supplementary Figure 6 and Supplementary Table ?? is reported
again the test set. We find human error in hand labelling reduced final precision to approximately 98.6%." 

On the confidence column only in that file: "Sentinel-2 and SPOT pipeline branches are combined into a final
vector dataset using a rules-based filter. Where Sentinel-2 and SPOT polygons intersect
with a Jaccard index (Intersection-over-union) in excess of 30%, the geometry of the SPOT
polygon is retained, inheriting the installation date from the Sentinel-2 detection (confidence level “A”). Detections from only the SPOT and S2 branches are retained with confidence levels “B” and “C” respectively. Where the IoU does not exceed 30%, the union
of both geometries are retained, inheriting the S2 installation date (confidence “D”)." 

#### France West Europe PV Installations 2023

From [research publication](https://doi.org/10.1038/s41597-023-01951-4): 
```
The Git repository contains the raw crowdsourcing data and all the material necessary to re-generate our training dataset and technical validation.  
It is structured as follows: the raw subfolder contains the raw annotation data from the two annotation campaigns and the raw PV installations’ metadata.  
The replication subfolder contains the compiled data used to generate our segmentation masks.  
The validation subfolder contains the compiled data necessary to replicate the analyses presented in the technical validation section.
```

We will be using the `replication` subfolder to generate our PV polygons geojson file.

In [ ]:
# bespoke pre-processing for datsets not directly available in geojson or shapefile format
# parse the point or polygon json files with geopandas, transform raw polygons or points features into proper geometry for geojson conversion
from shapely.geometry import Polygon, Point, MultiPolygon
import json

# TODO: make function for processing of france json geometries

def france_eur_pv_preprocess(ds_metadata, ds_subdir, metadata_dir='raw', crs=None, geom_type='Polygon'):
    ds_dir = Path(ds_metadata['output_dir'])
    data_dir = ds_dir / ds_subdir
    metadata_file = 'raw-metadata_df.csv' if metadata_dir == 'raw' else 'metadata_df.csv'
    metadata_file = ds_dir / metadata_dir / metadata_file
    coords_file = "polygon-analysis.json" if geom_type == 'Polygon' else "point-analysis.json"
    # keep files that are in the specified subdir and have the above filename
    geom_files = [fpath for fpath in ds_metadata['files'] if fpath.startswith(data_dir) and fpath.endswith(coords_file)]
    crs = crs or 'EPSG:4326' # default to WGS84

    # load the metadata file
    metadata_df = pd.read_csv(metadata_file)
    print(f"Loaded '{metadata_file.split('/')[-1]}' with {len(metadata_df)} rows")

    # load into geopandas, inspect the data, and add metadata_df to separate pd dataframe
    raw_features = []
    for geom_file_path in geom_files:
        campaign_name = Path(geom_file_path).parent.name
        print(f"Processing {campaign_name} campaign...")
        
        with open(geom_file_path, 'r') as f:
            geom_data = json.load(f)
        feat_types = set([f['type'] for f in geom_data])
        print(f"Feature types: {feat_types}")
    
        for idx, feature_dict in enumerate(geom_data):
            # Skip empty dictionaries
            if not feature_dict:
                continue
            
            try:
                feature_id = feature_dict.get('id', idx) # Use index if ID is not present

                # extract geometry and coordinates
                if geom_type == 'Polygon':
                    # feat_dict = [{'polygons': [{'points': {'x': <px_coord>, 'y': <px_coord>}, ...}]}, ...]
                    coords = feature_dict['polygons']
                    if isinstance(coords, list) and len(coords) > 0:
                            # Handle multiple polygons
                            polygons = []
                            for poly_coords in coords:
                                if len(poly_coords) >= 3:  # Need at least 3 points for a polygon
                                    polygons.append(Polygon(poly_coords))
                            
                            if len(polygons) == 1:
                                geometry = polygons[0]
                            else:
                                geometry = MultiPolygon(polygons)
                                
                            # Create feature dictionary with properties
                            feature = {
                                'id': feature_id,
                                'campaign': campaign_name,
                                'geometry': geometry
                            }
                    raw_features.append(feature)
                elif geom_type == 'Point':
                    # feat_dict = [{'clicks': [{'@type': 'Point', 'x': <px_coord>, 'y': <px_coord>}, ...]}, ...]
                    coords = feature_dict['clicks']
                    if isinstance(coords, list) and len(coords) > 0:
                        points = []
                        for point_coords in coords:
                            if 'x' not in point_coords or 'y' not in point_coords:
                                continue
                            else:
                                points.append(Point(point_coords['x'], point_coords['y']))
                    raw_features.extend(points)
            except Exception as e:
                print(f"Error processing feature {feature_dict}: {e}")
                continue

    if raw_features:
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(raw_features, crs=crs)
        # add metadata to the gdf
        if 'id' in gdf.columns:
            gdf['id'] = gdf['id'].astype(str)
        # Ensure CRS is set
        if gdf.crs is None:
            gdf.set_crs(crs, inplace=True)
        elif str(gdf.crs) != crs:
            gdf = gdf.to_crs(crs)
        # need to add geotransform if available to convert pixel coords to lat/lon

        # gdf['source_dataset'] = add in calling function
    
    return gdf, metadata_df

### Manual dataset file curation

In [ ]:
# keep subset of metadata dict for selected datasets
selected_metadata = {ds: dataset_metadata[ds] for ds in selected_datasets}
get_ds_files = lambda ds: dataset_metadata[ds]['files']
get_ds_dir = lambda ds: dataset_metadata[ds]['output_dir']
is_ds_ftype = lambda ds, fname: fname.endswith(f".{dataset_metadata[ds]['label_fmt']}")
get_full_ds_path = lambda ds: DATASET_DIR / 'raw' / 'labels' / ds
fra_ds_folder = 'replication'

# TODO: refactor this to a function as it'll quickly get out of hand with more datasets and pruning required
# make a manual selection of the set of files we'll use from each dataset
selected_ds_files = {ds : [f for f in get_ds_files(ds) if is_ds_ftype(ds, f)] for ds in selected_datasets}

# ad hoc selection of files for testing (keep files that contain 'solar' and 'WGS84' in filename)
selected_ds_files['global_harmonized_large_solar_farms_2020'] = [f for f in selected_ds_files['global_harmonized_large_solar_farms_2020'] if 'solar' in f.split('/')[-1] and 'WGS84' in f and not os.path.isdir(f)]
# prediction dataset was human verified thoroughly and meant for downstream applications; only use this file for now
selected_ds_files['global_pv_inventory_sent2_spot_2021'] = [f for f in selected_ds_files['global_pv_inventory_sent2_spot_2021'] if 'predicted' in os.path.basename(f)]
print(f"Selected {len(selected_ds_files['global_pv_inventory_sent2_spot_2021'])} files for {selected_datasets[0]}:\n{selected_ds_files['global_pv_inventory_sent2_spot_2021']}")

# only include files that were not filtered out
include_files = [os.path.basename(f) for ds in selected_datasets for f in selected_ds_files[ds]]
# don't print out unused directories
exclude_folders = [os.path.basename(dir) for dir in os.listdir(DATASET_DIR / 'raw' / 'labels') if dir not in selected_datasets]

# build and output tree for selected datasets
selected_ds_dirs = [get_ds_dir(ds) for ds in selected_datasets]
print("All selected datasets have been fetched with the following file tree:\n")
# TODO: fix unwanted dirs in the tree
selected_ds_tree = seedir(DATASET_DIR / 'raw' / 'labels', depthlimit=10, printout=True, regex=False, include_files=include_files, exclude_folders=exclude_folders)

# Dataset Processing with GeoPandas and Intro to GeoParquet

<div style="max-width: 60%; margin: 0 auto; padding-left: 2em; padding-right: 2em; text-align: justify left;">
    <p>GeoParquet is <a href="https://geoparquet.org/">an incubating Open Geospatial Consortium (OGC) standard</a> that simply adds compatible geospatial <a href="https://docs.safe.com/fme/html/FME-Form-Documentation/FME-ReadersWriters/geoparquet/Geometry-Support.htm">geometry types</a> (MultiPoint, Line, Polygon, etc) to the mature and widely adopted <a href="https://parquet.apache.org/">Apache Parquet format</a>, a popular columnar storage format commonly used in big data processing and modern data engineering pipelines and analytics. Despite the geoparquet standard only just recently reaching a v1.X release, Parquet itself is a very mature file format and has a wide ecosystem that GeoParquet seamlessly integrates with. This is analogous to how the GeoTIFF raster format adds geospatial metadata to the longstanding TIFF standard. GeoParquet is designed to be a simple and efficient way to store geospatial <em>vector</em> data in a columnar format, and is designed to be compatible with existing Parquet tools and libraries to enable Cloud <em>Data Warehouse</em> Interoperability.</p>
    <p>Parquet is a columnar storage format that is optimized for <strong>analytical workloads</strong> (i.e. not <strong>transactional</strong>) and is designed to work well with large-scale data processing frameworks like Apache Spark, Dask, and Apache Airflow. It is a <em>popular choice for storing large datasets using modern cloud-centric DBMS architectures</em> like data lakes and data warehouses, as it allows for multiple optimizations and efficient querying and analysis of data. Parquet files are <strong>designed to be highly compressed</strong>, which reduces storage costs and improves performance when reading and writing data. The columnar format allows for efficient compression algorithms to be <em>applied to each column independently</em>, resulting in better compression ratios compared to traditional row-based formats like CSV or JSON.</p>
    <p>These files are organized in a set of file chunks called "row groups". Row groups are logical groups of columns with the same number of rows. Each of these columns is actually a "column chunk" which is a contiguous block of data for that column. The schema across row groups must be consistent, i.e. the data types and number of columns must be the same for every row group. The new geospatial standard adds some relevant additional metadata such as the geometry's Coordinate Reference System (CRS), additional metadata for geometry columns, and recent releases have enabled <a href="https://medium.com/radiant-earth-insights/geoparquet-1-1-coming-soon-9b72c900fbf2">initial support for spatial indexing in v1.1</a>. <a href="https://towardsdatascience.com/geospatial-data-engineering-spatial-indexing-18200ef9160b">Spatial indexing</a> is a technique used to optimize spatial queries by indexing or partitioning the data based on its geometry features such that you can make spatial queries (e.g. intersection, within, within x distance, etc) more efficiently.</p>
</div>
  
  

<figure style="text-align: center">
<img src="https://miro.medium.com/v2/resize:fit:1400/1*QEQJjtnDb3JQ2xqhzARZZw.png" style="width:50%; height:auto;">
<figcaption align = "center"> Visualization of the layout of a Parquet file </figcaption>
</figure>

Beyond the file data itself, Parquet also stores metadata at the end of the file that describes the internal "chunking" of the file, byte ranges of every column chunks, several column statistics, among other things. 

<figure style="text-align: center">
<img src="https://guide.cloudnativegeo.org/images/geoparquet_layout.png" style="width:50%; height:auto;">
<figcaption align = "center"> GeoParquet has the same laylout with additional metadata </figcaption>
</figure>
  
  

## Features and Advantages

- Efficient storage and compression: 
    - leverages the columnar data format which is more efficient for filtering on columns
    - GeoParquet is internally compressed by default, and can be configured to optimize decompression time or storage size depending on the use case
    - These make it ideal for applications dealing with _massive_ geospatial datasets and cloud data warehouses
- Scalability and High-Performance:
    - the nature of the file format is well-suited for parallel and/or distributed processing such as in Spark, Dask, or Hadoop
    - Support for data partitioning: 
        - Parquet files can be partitioned by one or more columns
        - In the geospatial context this enables efficient spatial queries and filtering (e.g. partitioning by ISO country code) 
        - Properly implemented spatial partitions can enable [predicate pushdown](https://medium.com/radiant-earth-insights/geoparquet-1-1-coming-soon-9b72c900fbf2) which can significantly speed up spatial queries over the network by applying filters at the storage level and greatly reducing the amount of data that needs to be transferred. See this impressive example from the [geoparquet v1.1 blog](https://medium.com/radiant-earth-insights/geoparquet-1-1-coming-soon-9b72c900fbf2)
- Optimized for *read-heavy workflows*: 
    - Parquet is an immutable file format, which means taking advantage of cheap reads, and efficient filtering and aggregation operations
        - This is ideal for data warehousing and modern analytic workflows 
        - Best paired with Analytical Databases like Amazon Redshift, Google BigQuery, or DuckDB
        - Ideal for OLAP (Online Analytical Processing) and BI (Business Intelligence) workloads that leverage historical and aggregated data that don't require frequent updates
 - Interoperability and wide ecosystem:
    - GeoParquet is designed to be compatible with existing Parquet readers, tools, and libraries
    - Facilitates integration into existing data pipelines and workflows
    - Broad compatibility:
        - support for multiple spatial reference systems 
        - support for multiple geometry types and multiple geometry columns
        - works with both planar and spherical coordinates 
        - support for 2D and 3D geometries
        
## Limitations and Disadvantages

- Poorly suited for write-heavy workflows:
    - Transactional and CRUD (Create, Read, Update, Delete) operations are not well-suited for Parquet files
    - Not recommended for applications that require frequent updates or real-time data ingestion
- Not a Silver Bullet for all geospatial data:
    - deals only with vector data, not raster data
    - storage and compression benefits require a certain scale of data to be realized
    - performance overhead for small datasets
- Limited support for spatial indexing:
    - GeoParquet did not implement spatial indexing in the 1.0.0 release
    - This is planned to be built-in as a BBOX struct field (4 coords in a single column) for future release in version 1.1 of the standard

### GeoPandas Processing

In [ ]:
OVERLAP_THRESH = float(os.getenv('GEOM_OVERLAP_THRESHOLD', 0.8))
# additional preprocessing specific to each dataset (mostly attaching any included metadata)
def global_pv_inventory_spot_processing(gdf, dataset_name, output_dir, subset_bbox=None, geom_type='Polygon', rm_invalid=True):
    all_cols = [
        'unique_id', 'area', 'confidence', 'install_date', 'iso-3166-1', 'iso-3166-2', 'gti', 'pvout', 'capacity_mw', 'match_id', 'wdpa_10km', 'LC_CLC300_1992', 'LC_CLC300_1993',
        'LC_CLC300_1994', 'LC_CLC300_1995', 'LC_CLC300_1996', 'LC_CLC300_1997', 'LC_CLC300_1998', 'LC_CLC300_1999', 'LC_CLC300_2000', 'LC_CLC300_2001', 'LC_CLC300_2002',
        'LC_CLC300_2003', 'LC_CLC300_2004', 'LC_CLC300_2005', 'LC_CLC300_2006', 'LC_CLC300_2007', 'LC_CLC300_2008', 'LC_CLC300_2009', 'LC_CLC300_2010', 'LC_CLC300_2011',
        'LC_CLC300_2012', 'LC_CLC300_2013', 'LC_CLC300_2014', 'LC_CLC300_2015', 'LC_CLC300_2016', 'LC_CLC300_2017', 'LC_CLC300_2018', 'mean_ai', 'GCR', 'eff', 'ILR',
        'area_error', 'lc_mode', 'lc_arid', 'lc_vis', 'geometry', 'aoi_idx', 'aoi', 'id', 'Country', 'Province', 'WRI_ref', 'Polygon Source', 'Date', 'building',
        'operator', 'generator_source', 'amenity', 'landuse', 'power_source', 'shop', 'sport', 'tourism', 'way_area', 'access', 'denomination', 'historic',
        'leisure', 'man_made', 'natural', 'ref', 'religion', 'surface', 'z_order', 'layer', 'name', 'barrier', 'addr_housenumber', 'office', 'power',  'military'
    ]
    # these are not included in the prediction_set geojson: ['osm_id', 'Project', 'construction']
    # remove unwanted columns
    keep_cols = ['geometry', 'unique_id', 'confidence', 'install_date', 'capacity_mw', 'iso-3166-2', 'pvout']
    # keep_cols = all_cols
    print(f"Filtering from {len(all_cols)} columns to {len(keep_cols)} columns:\n{keep_cols}")
    gdf = gdf[keep_cols]
    return gdf
def global_pv_inventory_sent2_processing(gdf, dataset_name, output_dir, subset_bbox=None, geom_type='Polygon', rm_invalid=True):
    return gdf
def india_pv_solar_farms_processing(gdf, dataset_name, output_dir, subset_bbox=None, geom_type='Polygon', rm_invalid=True):
    return gdf
def usa_cali_usgs_pv_processing(gdf, dataset_name, output_dir, subset_bbox=None, geom_type='Polygon', rm_invalid=True):
    return gdf
def usa_eia_large_scale_pv_processing(gdf, dataset_name, output_dir, subset_bbox=None, geom_type='Polygon', rm_invalid=True):
    return gdf
def global_harmonized_large_solar_farms_processing(gdf, dataset_name, output_dir, subset_bbox=None, geom_type='Polygon', rm_invalid=True):
    all_cols = ['sol_id', 'GID_0', 'panels', 'panel.area', 'landscape.area', 'urban', 'power', 'geometry']
    # remove unwanted columns
    keep_cols = ['sol_id', 'panels', 'panel.area', 'landscape.area', 'water', 'urban', 'power', 'geometry']
    print(f"Filtering from {len(all_cols)} columns to {len(keep_cols)} columns:\n{keep_cols}")
    gdf = gdf[keep_cols]
    return gdf

def filter_gdf_duplicates(gdf, geom_type='Polygon', overlap_thresh=OVERLAP_THRESH):
    """
    Remove duplicate geometries from a GeoDataFrame based on a specified overlap threshold,
    keeping the geometry with the smaller area when two overlap substantially.
    
    Args:
        gdf (GeoDataFrame): Input GeoDataFrame.
        geom_type (str): Geometry type to filter by. Default is 'Polygon'.
        overlap_thresh (float): Overlap threshold for removing duplicates. Default is 0.8.
        
    Returns:
        gdf (GeoDataFrame): GeoDataFrame with duplicates removed.
    """
    # First identify exact duplicates
    gdf = gdf.drop_duplicates('geometry')
    
    # Identify geometries that overlap substantially
    overlaps = []
    # Use spatial index for efficiency
    spatial_index = gdf.sindex
    
    for idx, geom in enumerate(gdf.geometry):
        # Find potential overlaps using the spatial index
        possible_matches = list(spatial_index.intersection(geom.bounds))
        # Remove self from matches
        if idx in possible_matches:
            possible_matches.remove(idx)
        
        for other_idx in possible_matches:
            other_geom = gdf.iloc[other_idx].geometry
            if geom.intersects(other_geom):
                # Calculate overlap percentage (relative to the smaller polygon)
                intersection_area = geom.intersection(other_geom).area
                min_area = min(geom.area, other_geom.area)
                overlap_percentage = intersection_area / min_area if min_area > 0 else 0.0
                
                # If overlap is significant (e.g., > threshold)
                if overlap_percentage > overlap_thresh:
                    # Keep the geometry with the smaller area (presumably more precise)
                    if geom.area < other_geom.area:
                        overlaps.append(other_idx)
                        break
                    else:
                        overlaps.append(idx)
    
    # Remove overlapping geometries
    if overlaps:
        print(f"Removing {len(overlaps)} geometries with >{overlap_thresh*100}% overlap")
        gdf = gdf.drop(gdf.index[overlaps]).reset_index(drop=True)
    
    return gdf

# basic processing for geojson, shapefiles, and already georeferenced data
def process_vector_geoms(
    geom_files, 
    dataset_name, 
    output_dir=None, 
    subset_bbox=None, 
    geom_type='Polygon', 
    rm_invalid=True, 
    dedup_geoms=False,
    overlap_thresh=OVERLAP_THRESH, 
    out_fmt='geoparquet'):
    """
    Process a GeoJSON file and return a GeoDataFrame.
    
    Args:
        file_path (str): Path to the GeoJSON file.
        dataset_name (str): Name of the dataset.
        geom_type (str): Geometry type to filter by. Default is 'Polygon'.
        
    Returns:
        gdf (GeoDataFrame): Processed GeoDataFrame.
    """
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    ds_dataframes = []

    for fname in geom_files:
        # TODO: change to use geofile_regex
        if fname.endswith('.geojson') or fname.endswith('.json') or fname.endswith('.shp') or fname.endswith('.gpkg'):
            # Check if the file is a valid GeoJSON
            try:
                gdf = gpd.read_file(fname)
            except Exception as e:
                print(f"Error reading {os.path.relpath(fname)}: {e}")
                continue
            ds_dataframes.append(gdf)
    
    if len(ds_dataframes) == 0:
        print(f"No valid GeoJSON files found in {dataset_name}.")
        print(f"Skipping dataset {dataset_name}")
        return None
        
    # Concatenate all dataframes into a single GeoDataFrame
    gdf = gpd.GeoDataFrame(pd.concat(ds_dataframes, ignore_index=True))
    # make sure the geometry column is included and named correctly
    if 'geometry' not in gdf.columns:
        gdf['geometry'] = gdf.geometry

    # Basic info about the dataset
    print(f"Loaded geodataframe with raw counts of {len(gdf)} PV installations")
    print(f"Coordinate reference system: {gdf.crs}")
    print(f"Available columns: {gdf.columns.tolist()}")
    
    # Add dataset name as a new column
    gdf['dataset'] = dataset_name
    
    # Convert to WGS84 if not already in that CRS
    if gdf.crs is not None and gdf.crs.to_string() != 'EPSG:4326':
        # convert to WGS84 in cases of other crs (eg NAD83 for Cali dataset)
        gdf = gdf.to_crs(epsg=4326)
    if subset_bbox is not None:
        # Filter the GeoDataFrame by the georeferenced bounding box
        gdf = gdf.cx[subset_bbox[0]:subset_bbox[2], subset_bbox[1]:subset_bbox[3]]
    
    # DQ and cleaning
    # check for missing and invalid geometries
    invalid_geoms = gdf[gdf.geometry.is_empty | ~gdf.geometry.is_valid]
    if len(invalid_geoms) > 0 and rm_invalid:
        print(f"Warning: {len(invalid_geoms)} invalid or empty geometries found and will be removed.")
        # Optionally remove invalid geometries
        gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.is_valid].reset_index(drop=True)
    # Eliminating duplicates and geometries that overlap too much
    if geom_type == 'Polygon' and dedup_geoms:
        gdf = filter_gdf_duplicates(gdf, geom_type=geom_type, overlap_thresh=overlap_thresh)

    # perform any additional processing specific to the dataset for metadata and other attributes
    if dataset_name == 'global_pv_inventory_sent2_2024':
        print("Processing global_pv_inventory_sent2_2024 metadata")
        gdf = global_pv_inventory_sent2_processing(gdf, dataset_name, output_dir, subset_bbox=subset_bbox, geom_type=geom_type)
    elif dataset_name == 'global_pv_inventory_sent2_spot_2021':
        print("Processing global_pv_inventory_sent2_spot_2021 metadata")
        gdf = global_pv_inventory_spot_processing(gdf, dataset_name, output_dir, subset_bbox=subset_bbox, geom_type=geom_type)
    elif dataset_name == 'ind_pv_solar_farms_2022':
        print("Processing ind_pv_solar_farms_2022 metadata")
        gdf = india_pv_solar_farms_processing(gdf, dataset_name, output_dir, subset_bbox=subset_bbox, geom_type=geom_type)
    elif dataset_name == 'usa_cali_usgs_pv_2016':
        print("Processing usa_cali_usgs_pv_2016 metadata")
        gdf = usa_cali_usgs_pv_processing(gdf, dataset_name, output_dir, subset_bbox=subset_bbox, geom_type=geom_type)
    elif dataset_name == 'usa_eia_large_scale_pv_2023':
        print("Processing usa_eia_large_scale_pv_2023 metadata")
        gdf = usa_eia_large_scale_pv_processing(gdf, dataset_name, output_dir, subset_bbox=subset_bbox, geom_type=geom_type)
    
    # Re-project to a projected CRS (e.g. EPSG:3857) for accurate area calculations.
    # EPSG:4326 is a geographic CRS (lat/lon in degrees) and is not suitable
    # for calculating areas in square meters.
    gdf_proj = gdf.geometry.to_crs(epsg=3857)
    gdf['area_m2'] = gdf_proj.area

    gdf['centroid_lon'] = gdf.geometry.centroid.x
    gdf['centroid_lat'] = gdf.geometry.centroid.y
    # Optionally, you can also compute bounding boxes:
    # gdf['bbox'] = gdf.geometry.apply(lambda geom: geom.bounds)

    print(f"After filtering and cleaning, we have {len(gdf)} PV installations")
    print(f"Coordinate reference system: {gdf.crs}")
    print(f"Available columns: {gdf.columns.tolist()}")

    if output_dir:
        out_path = os.path.join(output_dir, f"{dataset_name}_processed.{out_fmt}")

        if out_fmt == 'geoparquet':
            gdf.to_parquet(out_path, 
                index=None, 
                compression='snappy',
                geometry_encoding='WKB', 
                write_covering_bbox=True,
                schema_version='1.1.0')
        else:
            # geopackage, shapefile, or geojson
            fmt_driver_map = {'geojson': 'GeoJSON', 'shp': 'ESRI Shapefile', 'gpkg': 'GPKG'}
            gdf.to_file(out_path, driver=fmt_driver_map[out_fmt], index=None)
        print(f"Saved processed GeoDataFrame to {os.path.relpath(out_path)}")
    
    return gdf


### Convert to GeoParquet

In [ ]:
random.shuffle(selected_datasets)
# go through the selected datasets and process them
for ds in selected_datasets:
    ds_files = selected_ds_files[ds]
    ds_dir = get_ds_dir(ds)
    ds_tree = dataset_metadata[ds]['tree']
    out_dir = DATASET_DIR / 'raw' / 'labels' / 'geoparquet'
    print(f"Processing dataset {ds} with {len(ds_files)} files in {os.path.relpath(ds_dir)}:\n{ds_files}")
    ds_gdf = process_vector_geoms(
                geom_files=ds_files,
                dataset_name=ds,
                output_dir=out_dir
    )
    if ds_gdf is not None:
        
        display(ds_gdf.describe())
        # print(ds_gdf.info)
        display(ds_gdf.sample(10))
    # remove gdf from memory as we only neeeded it for conversion and brief inspection
    del ds_gdf

In [ ]:
# columns for datasts
# ind_pv_solar_farms_2022 = ['State', 'Area', 'Latitude', 'Longitude', 'fid', 'geometry', 'dataset', 'area_m2', 'centroid_lon', 'centroid_lat']
# global_harmonized_large_solar_farms_2020 =  ['sol_id', 'GID_0', 'panels', 'panel.area', 'landscape.area', 'water', 'urban', 'power', 'geometry']
# usa_cali_usgs_pv_2016 = ['polygon_id', 'centroid_latitude', 'centroid_longitude', 'centroid_latitude_pixels', 'centroid_longitude_pixels', 'city', 'area_pixels', 'area_meters', 'image_name', 'nw_corner_of_image_latitude', 'nw_corner_of_image_longitude', 'se_corner_of_image_latitude', 'se_corner_of_image_longitude', 'datum', 'projection_zone', 'resolution', 'jaccard_index', 'polygon_vertices_pixels', 'geometry']
# global_pv_inventory_sent2_spot_2021 = ['unique_id', 'area', 'confidence', 'install_date', 'iso-3166-1', 'iso-3166-2', 'gti', 'pvout', 'capacity_mw', 'match_id', 'wdpa_10km', 'LC_CLC300_1992', 'LC_CLC300_1993', 'LC_CLC300_1994', 'LC_CLC300_1995', 'LC_CLC300_1996', 'LC_CLC300_1997', 'LC_CLC300_1998', 'LC_CLC300_1999', 'LC_CLC300_2000', 'LC_CLC300_2001', 'LC_CLC300_2002', 'LC_CLC300_2003', 'LC_CLC300_2004', 'LC_CLC300_2005', 'LC_CLC300_2006', 'LC_CLC300_2007', 'LC_CLC300_2008', 'LC_CLC300_2009', 'LC_CLC300_2010', 'LC_CLC300_2011', 'LC_CLC300_2012', 'LC_CLC300_2013', 'LC_CLC300_2014', 'LC_CLC300_2015', 'LC_CLC300_2016', 'LC_CLC300_2017', 'LC_CLC300_2018', 'mean_ai', 'GCR', 'eff', 'ILR', 'area_error', 'lc_mode', 'lc_arid', 'lc_vis', 'geometry', 'key', 'resolution', 'pad', 'tilesize', 'zone', 'cs_code', 'ti', 'tj', 'proj4', 'wkt', 'ISO_A2', 'ISO_A3', 'idx', 'aoi_idx', 'aoi', 'id', 'Country', 'Province', 'Project', 'WRI_ref', 'Polygon Source', 'Date', 'building', 'operator', 'generator_source', 'amenity', 'landuse', 'power_source', 'shop', 'sport', 'tourism', 'way_area', 'access', 'construction', 'denomination', 'historic', 'leisure', 'man_made', 'natural', 'ref', 'religion', 'surface', 'z_order', 'layer', 'name', 'barrier', 'addr_housenumber', 'office', 'power', 'osm_id', 'military']

# could include power/capacity fields and leave as NULL for the others
# consolidated = [unify_ids(fid, sol_id/GID_0, polygon_id, unique_id), 'area_m2', 'centroid_lon', 'centroid_lat', 'dataset', 'geometry', 'bbox']

### DuckDB and/or GeoPandas dataset consolidation

In [ ]:
import os
import duckdb
from typing import Optional, List

def geom_db_consolidate_dataset(
    parquet_files: List[str],
    out_db_file: str,
    table_name: str = "global_consolidated_pv",
    geom_column: str = "geometry",
    spatial_index: bool = True,
    out_parquet: Optional[str] = None,
    printout: bool = False
):
    """
    Read a list of GeoParquet files into DuckDB, keeps only a consolidated list 
    of columns, union these geoparquets into a single table, and filters out duplicate
    geometries based on spatial overlap. 
    
    For each file, the following columns are selected:
      - unified_id: md5 hash of (COALESCE(fid, sol_id, GID_0, polygon_id, unique_id) + dataset)
      - area_m2
      - centroid_lon
      - centroid_lat
      - dataset (derived from the filename)
      - bbox: bounding box of the geometry written by geopandas during export to geoparquet
      - geometry (converted with ST_GeomFromWKB)

    Args:
        parquet_files: list of paths to input parquet files.
        out_db_file: path to DuckDB database file.
        table_name: name of the consolidated table in DuckDB.
        geom_column: name of the geometry column in the parquet files.
        spatial_index: if True, create a spatial index on the geometry column.
        out_parquet: optional path to write consolidated table out as a GeoParquet file.
        printout: if True, prints statistics (counts, area distribution, bounding box).
    """
    # remove any existing database file (temporary workaround for failing to load existing db file before loading spatial extension)
    if os.path.exists(out_db_file):
        os.remove(out_db_file)

    print(f"Connecting to DuckDB database: {out_db_file}")
    conn = duckdb.connect(database=out_db_file, read_only=False)
    conn.install_extension("spatial")
    conn.load_extension("spatial")
    
    # drop any existing table
    conn.execute(f"DROP TABLE IF EXISTS {table_name};")
    
    scans = []
    print(f"Building temp UNION query from {len(parquet_files)} parquet files...")
    for path in parquet_files:
        print(f"  Processing file: {os.path.basename(path)}")
        # Derive a dataset name from the parquet file's basename
        dataset_name = os.path.basename(str(path)).split('.')[0]
        import pyarrow.parquet as pq

        # Read the parquet schema to get available column names
        schema = pq.read_schema(str(path))
        cols = set(schema.names)
        required_cols = {'area_m2', 'centroid_lon', 'centroid_lat', 'bbox', geom_column}
        # print(f"Available columns in {dataset_name}: \n{cols}")

        # Check for required columns
        missing_cols = required_cols - cols
        if missing_cols:
            print(f"    WARNING: Skipping file {path_str}. Missing required columns: {missing_cols}")
            continue
        
        uid_candidates = ['fid', 'sol_id', 'GID_0', 'polygon_id', 'unique_id']
        present_uids = [f"CAST({uid} AS VARCHAR)" for uid in uid_candidates if uid in cols]

        # If none of the uid columns are present, use NULL
        if not present_uids:
            present_uids = ["NULL"]

        uid_expr = "COALESCE(" + ", ".join(present_uids) + ", 'NO_ID_' || ROW_NUMBER() OVER ())" # Add fallback

        # TODO: select all columns and still perform geometry conversion
        scan_query = f"""
            SELECT 
              md5(concat_ws('_', {uid_expr}, '{dataset_name}')) AS unified_id,
              area_m2,
              centroid_lon,
              centroid_lat,
              '{dataset_name}' AS dataset,
              bbox, 
              ST_GeomFromWKB(TRY_CAST({geom_column} AS WKB_BLOB)) AS geometry
            FROM read_parquet('{str(path)}')
        """
        scans.append(scan_query)

    if not scans:
        print("No valid parquet files found. Exiting.")
        conn.close()
        return None, None
    
    union_sql = "\nUNION ALL\n".join(scans)
    # create a temporary table with the unioned results
    print("Creating temporary table 'tmp' with combined data...")
    try:
        conn.execute(f"CREATE TEMPORARY TABLE tmp AS {union_sql};")
        count_tmp = conn.execute("SELECT COUNT(*) FROM tmp").fetchone()[0]
        print(f"Temporary table 'tmp' created with raw count (before dedup) of {count_tmp} records.")
        if count_tmp == 0:
            print("ERROR: Temporary table is empty after UNION. Check input files and queries.")
            conn.close()
            return None, None
    except Exception as e:
        print(f"ERROR creating temporary table: {e}")
        print("UNION SQL used:")
        print(union_sql)
        conn.close()
        return None, None

    t1, t2 = None, None
    # optionally create a spatial index on the temporary table
    if spatial_index:
        t1 = time.time()
        conn.execute(f"CREATE INDEX tmp_pv_idx ON tmp USING RTREE (geometry);")
        t2 = time.time()
        print(f"Spatial index created on temporary table (used for dedup) in {t2 - t1:.2f} seconds.")
    
    print(f"Creating final table '{table_name}' by filtering duplicates...")
    
    # add a unique row identifier within the temp table in edge case where unified_id is not unique
    cte1 = f"""
        SELECT
            ROW_NUMBER() OVER () as temp_row_id,
            *
        FROM tmp
    """

    # select distinct geometries first to reduce comparisons if exact duplicates exist
    # this assumes unified_id might not be unique per geometry (initially!) 
    cte2 = f"""
        SELECT DISTINCT ON (geometry) *
        FROM NumberedSource
        ORDER BY geometry, unified_id -- Ensure deterministic selection for exact duplicates
    """

    # filter out duplicates based on spatial intersection
    # dedup_query = f"""
    # CREATE TABLE {table_name} AS
    # WITH NumberedSource AS (
    #     {cte1}
    # ), IndexedSource AS (
    #     {cte2}
    # )
    
    # SELECT DISTINCT ON (a.temp_row_id) a.* EXCLUDE (temp_row_id) -- Select distinct rows based on temp_row_id
    # FROM IndexedSource a
    # LEFT JOIN IndexedSource b
    # ON a.temp_row_id != b.temp_row_id -- Ensure not comparing a row to itself
    #    AND a.unified_id > b.unified_id -- Deterministic way to pick which one to keep
    #    AND ST_Intersects(a.geometry, b.geometry)
    #    AND ST_Area(ST_Intersection(a.geometry, b.geometry)) / ST_Area(a.geometry) > {OVERLAP_THRESH}
    # WHERE b.temp_row_id IS NULL; -- Keep 'a' if no 'better' overlapping geometry 'b' was found
    # """

    # V2
    # dedup_query = f"""
    #     CREATE TABLE {table_name} AS
    #     WITH TmpWithId AS (
    #         -- Add row_id to the already filtered tmp table for reliable self-join
    #         SELECT ROW_NUMBER() OVER () as temp_row_id, * FROM tmp
    #     ), OverlapPairs AS (
    #         -- Identify pairs (a, b) where 'a' significantly overlaps 'b', and 'a' has a lower unified_id
    #         SELECT
    #             b.temp_row_id as b_id_to_remove -- Select the ID of the row to potentially remove
    #         FROM TmpWithId a
    #         JOIN TmpWithId b ON a.temp_row_id != b.temp_row_id -- Ensure not joining row to itself
    #             AND a.unified_id < b.unified_id -- Deterministic: 'a' is preferred over 'b'
    #             AND ST_Intersects(a.geometry, b.geometry) -- Use spatial index on tmp efficiently
    #             -- Check overlap relative to 'b's area. Assume ST_Area(b.geometry) > 0 due to initial filtering.
    #             AND (ST_Area(ST_Intersection(a.geometry, b.geometry)) / ST_Area(b.geometry)) > {OVERLAP_THRESH}
    #     ), RowsToRemove AS (
    #         -- Get the unique set of temp_row_ids that were identified as 'b' in OverlapPairs
    #         SELECT DISTINCT b_id_to_remove FROM OverlapPairs
    #     )
    #     -- Select rows from the temp table (with added ID) that are NOT marked for removal
    #     -- Then, handle any remaining *exact* geometry duplicates by keeping the one with the lowest unified_id
    #     SELECT DISTINCT ON (t.geometry) t.* EXCLUDE (temp_row_id)
    #     FROM TmpWithId t
    #     LEFT JOIN RowsToRemove r ON t.temp_row_id = r.b_id_to_remove
    #     WHERE r.b_id_to_remove IS NULL -- Keep only rows that are NOT in the removal set
    #     ORDER BY t.geometry, t.unified_id; -- Required for DISTINCT ON: keep the first unified_id for exact geom duplicates
    # """
    # backup if spatial dedup is much slower than geopandas filtering performed above
    dedup_query = f"""
        CREATE TABLE {table_name} AS
        SELECT DISTINCT ON (geometry) *
        FROM tmp
        ORDER BY geometry, unified_id; -- Required for DISTINCT ON: keep the first unified_id for exact geom duplicates
    """

    print("Executing deduplication query...")
    # print(f"Deduplication Query:\n{dedup_query}") # Uncomment for debugging
    t1 = time.time()
    conn.execute(dedup_query)
    t2 = time.time()
    print(f"Deduplication using spatial index and spatial functions completed in {t2 - t1:.2f} seconds.")
    count_final = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
    print(f"Final table '{table_name}' created with {count_final} deduplicated rows.")
    
    if spatial_index:
        t1 = time.time()
        conn.execute(f"CREATE INDEX {table_name}_geom_idx ON {table_name} USING RTREE (geometry);")
        t2 = time.time()
        print(f"Spatial index created on '{table_name}' in {t2 - t1:.2f} seconds.")
    
    if printout:
        # Area distribution (min, max, avg)
        area_stats = conn.execute(
            f"SELECT MIN(area_m2), MAX(area_m2), AVG(area_m2) FROM {table_name}"
        ).fetchone()
        # Bounding box based on centroidsx``
        bbox = conn.execute(
            f"SELECT MIN(centroid_lon), MAX(centroid_lon), MIN(centroid_lat), MAX(centroid_lat) FROM {table_name}"
        ).fetchone()
        print(f"Table '{table_name}' contains {count_final} records.")
        print(f"Area (m²): min = {area_stats[0]}, max = {area_stats[1]}, avg = {area_stats[2]}")
        print(f"Centroid extent: lon [{bbox[0]}, {bbox[1]}], lat [{bbox[2]}, {bbox[3]}]")
    
    # Optionally export consolidated table to geoparquet
    if out_parquet:
        # make sure the output directory exists
        os.makedirs(os.path.dirname(out_parquet), exist_ok=True)
        # DuckDB can export tables or query results to parquet using COPY
        conn.execute(f"COPY {table_name} TO '{out_parquet}' (FORMAT 'parquet', COMPRESSION 'snappy', ROW_GROUP_SIZE 10000);")
        print(f"Exported consolidated table to {out_parquet}")

    conn.close()
    print(f"Consolidated {len(parquet_files)} files into {out_db_file} → table '{table_name}'")
    if printout:
        print(f"Exported to: {out_parquet}")

    return out_db_file, table_name

In [ ]:
# get list of geoparquet files to be consolidated
get_full_gpq_path = lambda f: DATASET_DIR / 'raw' / 'labels' / 'geoparquet' / f
parquet_files = [get_full_gpq_path(f) for f in os.listdir(DATASET_DIR / 'raw' / 'labels' / 'geoparquet') if any(os.path.splitext(f)[0].startswith(ds) for ds in selected_datasets)]
flist = '\n-'.join([os.path.relpath(f) for f in parquet_files])
print(f"Consolidating these {len(parquet_files)} files:\n-{flist}")
out_db_dir = DATASET_DIR / 'prepared' / 'labels' 
out_consolidated_parquet = out_db_dir / 'geoparquet' / 'global_consolidated_pv.geoparquet'
out_consolidated_db = out_db_dir / 'db' / 'global_consolidated_pv.duckdb'
# create the output directories if they don't exist
print(f"Creating output directories: {out_consolidated_db.parent}") 
os.makedirs(out_consolidated_db.parent, exist_ok=True)
print(f"Creating output directories: {out_consolidated_parquet.parent}")
os.makedirs(out_consolidated_parquet.parent, exist_ok=True)
# consolidate the dataset into a single duckdb database that will also be saved as a geoparquet file
db_file, table_name = geom_db_consolidate_dataset(
    parquet_files=parquet_files,
    out_db_file=out_consolidated_db,
    table_name="global_consolidated_pv",
    geom_column="geometry",
    spatial_index=True,
    out_parquet=out_consolidated_parquet,
    printout=True
)

In [ ]:
# load consolidated geoparquet into geodataframe to use in visualizations
ds_gdf = gpd.read_parquet(out_consolidated_parquet)

In [ ]:
display(ds_gdf.describe())
display(ds_gdf.sample(10))
ds = 'global_consolidated_pv'

In [ ]:
# perform geopandas dedup and check size again
ds_gdf = filter_gdf_duplicates(ds_gdf)
print(f"After filtering and cleaning, we have {len(ds_gdf)} PV installations")
display(ds_gdf.describe())
display(ds_gdf.sample(10))
# save the consolidated geoparquet file in our prepared directory
consolidated_dedup_parquet = out_db_dir / 'geoparquet' / 'global_consolidated_pv_dedup.geoparquet'
# drop existing bbox column to recalculate with reduced geometries
if 'bbox' in ds_gdf.columns:
    ds_gdf = ds_gdf.drop(columns=['bbox'])
# save the processed geoparquet file
ds_gdf.to_parquet(consolidated_dedup_parquet, 
    index=None, 
    compression='snappy',
    geometry_encoding='WKB', 
    write_covering_bbox=True,
    schema_version='1.1.0')

In [ ]:
# from maxar_platform.api_keys import generate_key
# key = generate_key("my new key")
# print(key['api_key'])

# Fetching vector layers for Spatio-temporal context

To better manage our *global* PV datasets for clustered STAC searches, we will also fetch some vector layers that enable us to organize the datasets by country, continent, and provide other spatio-temporal context relevant to our research.

**NOTE**: Fetching Overture divisions locally will result in a **~6.4GB file**!; Be aware that you can use blob storage with duckdb

In [ ]:
# 
# id	string	A feature ID. This may be an ID associated with the Global Entity Reference System (GERS) if—and-only-if the feature represents an entity that is part of GERS.
# geometry	blob	A WKB representation of the entity's geometry - a Point, Polygon, MultiPolygon, or LineString.
# bbox	array	The bounding box of an entity's geometry, represented with float values, in a xmin, xmax, ymin, ymax format.
# version	integer	Version number of the feature, incremented in each Overture release where the geometry or attributes of this feature changed.
# sources	array	The array of source information for the properties of a given feature, with each entry being a source object which lists the property in JSON Pointer notation and the dataset that specific value came from. All features must have a root level source which is the default source if a specific property's source is not specified.
# subtype	string	Category of the division from a finite, hierarchical, ordered list of categories (e.g. country, region, locality, etc.) similar to a Who's on First placetype.
# wikidata	string	A wikidata ID if available, as found on https://www.wikidata.org/.
# population	integer	Population of the division.
# names	array	A primary name of the entity, and a set of optional name translations. Name translations are represented in key, value pairs, where the key is an ISO language code and the value is the translated name.
# class	string	A value to represent whether an entity represents a maritime or land feature.
# division_ids	list	A list of the two division IDs that share this division boundary.
# is_disputed	boolean	Indicator if there are entities disputing this division boundary. Information about entities disputing this boundary should be included in perspectives property. This property should also be true if boundary between two entities is unclear and this is "best guess". So having it true and no perspectives gives map creators reason not to fully trust the boundary, but use it if they have no other.
# perspectives	array	Political perspectives from which this division boundary is considered to be an accurate representation. If this property is absent, then this boundary is not known to be disputed from any political perspective. Consequently, there is only one boundary feature representing the entire real world entity. If this property is present, it means the boundary represents one of several alternative perspectives on the same real-world entity.
# local_type	string	Local name for the subtype property, optionally localized. This property is localized using a standard Overture names structure.
# country	string	ISO 3166-1 alpha-2 country code of the country or country-like entity, that this division represents or belongs to. If the entity this division represents has a country code, the country property contains it. If it does not, the country property contains the country code of the first division encountered by traversing the parent_division_id chain to the root.
# region	string	ISO 3166-2 principal subdivision code of the subdivision-like entity this division represents or belongs to. If the entity this division represents has a principal subdivision code, the region property contains it. If it does not, the region property contains the principal subdivision code of the first division encountered by traversing the parent_division_id chain to the root.
# hierarchies	Array	Hierarchies in which this division participates.
# parent_division_id	string	Division ID of this division's parent division. Not allowed for top-level divisions (countries) and required for all other divisions. The default parent division is the parent division as seen from the default political perspective, if there is one, and is otherwise chosen somewhat arbitrarily. The hierarchies property can be used to inspect the exhaustive list of parent divisions.
# norms	list	Collects information about local norms and rules within the division that are generally useful for mapping and map-related use cases. If the norms property or a desired sub-property of the norms property is missing on a division, but at least one of its ancestor divisions has the norms property and the desired sub-property, then the value from the nearest ancestor division may be assumed.
# capital_division_ids	array	Division IDs of this division's capital divisions. If present, this property will refer to the division IDs of the capital cities, county seats, etc. of a division.
# capital_of_divisions	list	Division ID of the division that this feature is the capital of. If present, this property will refer to the division IDs of a parent county, region, country, etc.
# division_id	string	Division ID of the division this area belongs to.
# cartography	array	Contains a prominence property, which offers a suggestion for displaying Overture features at specific zoom levels based on it's importance and significance.
# is_land	boolean	Indicates whether or not the feature geometry represents the non-maritime "land" boundary, which can be used for map rendering, cartographic display, and similar purposes.
# is_territorial	boolean	Indicates whether or not the feature geometry represents the full territorial boundary or claim of a feature.
# filename	string	Name of the S3 file being queried.
# theme	string	Name of the Overture theme being queried.
# type	string	Name of the Overture feature type being queried.

# country
# dependency
# region
# county
# localadmin
# locality
# macrohood
# neighborhood
# subtype	

In [ ]:
# OVERTURE_DIVISION_AREA_S3 = os.getenv("OVERTURE_DIVISIONS_S3")

# # use duckdb to fetch overture's division geoparquets from s3 and save to our raw geoparquet labels

# def fetch_overture_division_areas(
#     s3_path: str,
#     out_gpq_path: str, # can be local or blob storage
#     s3_region: str = "us-east-1",
#     geom_col: str = "geometry",
#     subset_bbox: Optional[List[float]] = None,
# ):
# """
# Fetches Overture division area geoparquet files from S3 and saves them to a local path.
# """
#     print(f"Fetching Overture division areas from S3:\n {s3_path}")
#     print(f"WIll be saved to:\n {out_gpq_path}")

#     conn = None
#     try:
#         # use in0-memory connection since we'll only be copying the files over
#         conn = duckdb.connect(database=':memory:', read_only=True)
#         print("  Loading DuckDB extensions...")
#         conn.execute("INSTALL httpfs;")
#         conn.execute("LOAD httpfs;")
#         conn.execute("INSTALL spatial;")
#         conn.execute("LOAD spatial;")
#         # Configure S3 access (anonymous for public Overture data)
#         conn.execute("SET s3_use_ssl=true;")
#         conn.execute(f"SET s3_region='{s3_region}';") 

#         overture_file_cnt = conn.execute()
#         copy_sql = f"""
#         COPY (
#             SELECT
#                 *
#             FROM
#                 read_parquet('{s3_path}', hive_partitioning=1)
#         ) TO '{out_gpq_path}';
#         """
#         conn.execute(copy_sql)




# # and perform a spatial join with our polygon data
# def spatial_join_pv_overture_duckdb(
#     pv_gpq_path: str,
#     ov_divisions_gpq_path: str,
#     out_gpq_path: str,
#     pv_geom_col: str = "geometry",
#     overture_geom_col: str = "geometry",
#     join_predicate: str = "ST_Within", # Or ST_Intersects, ST_Contains etc.
#     overture_cols_to_keep: list[str] = ["id", "country", "region", "locality", "sublocality", "localityType", "isoCountryCodeAlpha2"]
# ):
#     """
#     Performs a spatial join between a local PV GeoParquet file and Overture
#     division areas directly from S3 using DuckDB.

#     Args:
#         pv_gpq_path: Path to the input GeoParquet file containing PV polygons.
#         ov_divisions_gpq_path: S3 glob path to the Overture division area Parquet files.
#         out_gpq_path: Path to save the resulting joined GeoParquet file.
#         pv_geom_col: Name of the geometry column in the PV data.
#         overture_geom_col: Name of the geometry column in the Overture data.
#         join_predicate: DuckDB spatial function for the join (e.g., 'ST_Within').
#         overture_cols_to_keep: List of columns to keep from the Overture data.
#     """
#     print(f"Starting DuckDB spatial join...")
#     print(f"  PV Data in: {os.ppqth(pv_geoparquet_path)}")
#     print(f"  Overture Data: {ov_divisions_gpq_path}")
#     print(f"  Output: {output_papqh}")

    

#         # Ensure output directory exists
#         os.makedirs(os.path.dirname(output_geoparquet_path), exist_ok=True)

#         print("  Constructing and executing SQL query...")

#         # Ensure overture columns are distinct if overlaps exist in pv data
#         select_overture_cols = ", ".join([f"ov.{col}" for col in overture_cols_to_keep])

#         # Assumes geometries are stored as WKB (common for GeoParquet)
#         # If not, adjust the ST_GeomFromWKB part accordingly
#         sql = f"""
#         COPY (
#             SELECT
#                 pv.*,
#                 {select_overture_cols}
#             FROm pqrquet('{pv_geoparquet_path}') AS pv
#             LEFT JOIN read_parquet('{ov_divisions_gpq_path}', hive_partitioning=1) AS ov
#             ONoipqte}(
#                 ST_GeomFromWKB(pv."{pv_geom_col}"),
#                 ST_GeomFromWKB(ov."{overture_geom_col}")
#             )
#         ) TO '{output_geoparquet_path}' (FORMAT 'PARQUET');
#         """
#         # print(f"  SQL Query:\n{sql}") # Uncomment to debug query
#         conn.execute(sql)
#         print(f"  Successfully executed join and saved results to {output_geoparquet_path}")

#     except Exception as e:
#         print(f"An error occurred during DuckDB join: {e}")
#     finally:
#         if conn:
#             conn.close()


# Visualization Functions for PV Data

After processing the datasets into standardized geoparquet format, we'll use the following visualization libraries to explore and present the data:

- **Folium**: For interactive web maps with various basemaps and markers
- **Pydeck**: For high-performance 3D and large-scale visualizations
- **Lonboard**: For GPU-accelerated geospatial visualization of large datasets

Each library has specific strengths that we'll leverage for different visualization needs.

## Folium Visualization Functions

Folium is excellent for creating interactive web maps with various basemaps and markers. It's particularly useful for visualizing geographic distributions and creating choropleth maps.

In [ ]:
from folium.plugins import HeatMap, MarkerCluster

def create_folium_cluster_map(gdf, zoom_start=19, title="PV Installation Clusters"):
    """
    Create a cluster map of PV installations using Folium.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame containing PV installation data with geometry column
    zoom_start : int
        Initial zoom level for the map
    title : str
        Title for the map
        
    Returns:
    --------
    folium.Map
        Interactive Folium map with clustered markers
    """
    # Ensure the GeoDataFrame is in WGS84 (EPSG:4326) for Folium compatibility
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Get centroid of all points to center the map
    center_lat = gdf.geometry.centroid.y.mean()
    center_lon = gdf.geometry.centroid.x.mean()
    
    # Create base map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start,
                  tiles='CartoDB positron')
    
    # Add title
    title_html = f'''
             <h3 align="center" style="font-size:16px"><b>{title}</b></h3>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add marker cluster
    marker_cluster = MarkerCluster().add_to(m)
    
    # Add markers for each PV installation
    for idx, row in gdf.iterrows():
        # Get the centroid if the geometry is a polygon
        if row.geometry.geom_type in ['Polygon', 'MultiPolygon']:
            centroid = row.geometry.centroid
            popup_text = f"ID: {idx}"
            
            # Add additional information if available in the dataframe
            for col in ['capacity_mw', 'area_m2', 'installation_date', 'dataset', 'cen']:
                if col in gdf.columns:
                    popup_text += f"<br>{col}: {row[col]}"
            
            folium.Marker(
                location=[centroid.y, centroid.x],
                popup=folium.Popup(popup_text, max_width=300),
                icon=folium.Icon(color='green', icon='solar-panel', prefix='fa')
            ).add_to(marker_cluster)
    
    return m

def create_folium_choropleth(gdf, column, bins=8, cmap='YlOrRd', 
                             title="PV Installation Density"):
    """
    Create a choropleth map of PV installations using Folium.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame containing PV installation data with geometry column
    column : str
        Column name to use for choropleth coloring
    bins : int
        Number of bins for choropleth map
    cmap : str
        Matplotlib colormap name
    title : str
        Title for the map
        
    Returns:
    --------
    folium.Map
        Interactive Folium choropleth map
    """
    # Ensure the GeoDataFrame is in WGS84 (EPSG:4326)
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Get centroid of all points to center the map
    center_lat = gdf.geometry.centroid.y.mean()
    center_lon = gdf.geometry.centroid.x.mean()
    
    # Create base map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=3,
                  tiles='CartoDB positron')
    
    # Add title
    title_html = f'''
             <h3 align="center" style="font-size:16px"><b>{title}</b></h3>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Create choropleth layer
    folium.Choropleth(
        geo_data=gdf,
        name='choropleth',
        data=gdf,
        columns=[gdf.index.name if gdf.index.name else 'index', column],
        key_on='feature.id',
        fill_color=cmap,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=column,
        bins=bins
    ).add_to(m)
    
    # Add hover functionality
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color': '#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color': '#000000', 
                                    'fillOpacity': 0.5, 
                                    'weight': 0.1}
    
    # Add tooltips
    folium.GeoJson(
        gdf,
        style_function=style_function,
        highlight_function=highlight_function,
        tooltip=folium.GeoJsonTooltip(
            fields=[column],
            aliases=[column.replace('_', ' ').title()],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
        )
    ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

def create_folium_heatmap(gdf, intensity_column=None, radius=15, 
                          title="PV Installation Heatmap"):
    """
    Create a heatmap of PV installations using Folium.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame containing PV installation data with geometry column
    intensity_column : str, optional
        Column name to use for heatmap intensity; if None, all points have equal weight
    radius : int
        Radius for heatmap points (in pixels)
    title : str
        Title for the map
        
    Returns:
    --------
    folium.Map
        Interactive Folium heatmap
    """
    # Ensure the GeoDataFrame is in WGS84 (EPSG:4326)
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Get centroids for all geometries
    if any(gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])):
        centroids = gdf.geometry.centroid
    else:
        centroids = gdf.geometry
    
    # Get coordinates for heatmap
    heat_data = [[point.y, point.x] for point in centroids]
    
    # Add intensity if specified
    if intensity_column and intensity_column in gdf.columns:
        heat_data = [[point.y, point.x, float(intensity)] 
                    for point, intensity in zip(centroids, gdf[intensity_column])]
    
    # Get centroid of all points to center the map
    center_lat = sum(point[0] for point in heat_data) / len(heat_data)
    center_lon = sum(point[1] for point in heat_data) / len(heat_data)
    
    # Create base map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=4,
                  tiles='CartoDB positron')
    
    # Add title
    title_html = f'''
             <h3 align="center" style="font-size:16px"><b>{title}</b></h3>
             '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Add heatmap layer
    HeatMap(
        heat_data,
        radius=radius,
        blur=10,
        gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}
    ).add_to(m)
    
    return m

In [ ]:
# filter to only rows without NaN for folium viz
# ds_gdf = ds_gdf[ds_gdf['area_m2'].notna()]
# Create Folium maps
cluster_map = create_folium_cluster_map(ds_gdf, title=f"{ds} - Cluster Map")
# cluster_map.save(os.path.join(out_dir, f"{ds}_cluster_map.html"))
cluster_map

In [ ]:
# create explicit index column like chloropleth expects
# ds_gdf.reset_index(drop=False, inplace=True)
# choropleth_map = create_folium_choropleth(ds_gdf, column='area_m2', title=f"{ds} - Capacity Choropleth")
# choropleth_map.save(os.path.join(out_dir, f"{ds}_choropleth_map.html"))

In [ ]:
# heatmap = create_folium_heatmap(ds_gdf, intensity_column='area_m2', title=f"{ds} - Capacity Heatmap")
# heatmap.save(os.path.join(out_dir, f"{ds}_heatmap.html"))
# heatmap

## PyDeck Visualization Functions

PyDeck is excellent for high-performance 3D visualizations and handling large datasets. It's particularly useful for creating layered maps with multiple types of data.

In [ ]:
# --- Helper Function for Color Mapping ---

def get_pydeck_color(df, color_column=None, cmap='viridis', default_color=[0, 128, 0, 180]):
    """
    Generates color mapping instructions for Pydeck layers.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        color_column (str, optional): Column to base color on. Defaults to None.
        cmap (str, optional): Matplotlib colormap name for numeric data. Defaults to 'viridis'.
        default_color (list, optional): Default RGBA color if no column specified. Defaults to green.

    Returns:
        tuple: (Updated DataFrame with color columns if needed, Pydeck color accessor string/list)
    """
    if not color_column or color_column not in df.columns:
        return df, default_color # Return default color list

    # Ensure color column NaNs are handled (replace with a placeholder or mean/median if appropriate)
    if df[color_column].isnull().any():
        print(f"Warning: NaN values found in color column '{color_column}'. Filling with placeholder/mean.")
        if pd.api.types.is_numeric_dtype(df[color_column]):
            fill_val = df[color_column].mean() # Or median, or 0
            df[color_column] = df[color_column].fillna(fill_val)
        else:
            fill_val = 'Unknown'
            df[color_column] = df[color_column].astype(str).fillna(fill_val)


    if pd.api.types.is_numeric_dtype(df[color_column]):
        # --- Numeric Column: Use a colormap ---
        try:
            from matplotlib import colormaps
            from matplotlib.colors import Normalize

            norm = Normalize(vmin=df[color_column].min(), vmax=df[color_column].max())
            colormap = colormaps[cmap]

            # Apply colormap and convert to RGBA 0-255 format
            colors = colormap(norm(df[color_column].values)) * 255
            df['__color_r'] = colors[:, 0].astype(int)
            df['__color_g'] = colors[:, 1].astype(int)
            df['__color_b'] = colors[:, 2].astype(int)
            # Use a fixed alpha or make it dynamic if needed
            df['__color_a'] = default_color[3] if len(default_color) == 4 else 180

            return df, '[__color_r, __color_g, __color_b, __color_a]'
        except ImportError:
            print("Matplotlib not found. Falling back to default color for numeric column.")
            return df, default_color
        except Exception as e:
            print(f"Error applying numeric colormap: {e}. Falling back to default color.")
            return df, default_color

    else:
        # --- Categorical Column: Assign random colors ---
        unique_cats = df[color_column].astype(str).unique()
        # Generate pseudo-random but consistent colors for categories
        color_map = {
            cat: [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), default_color[3] if len(default_color) == 4 else 180]
            for cat in unique_cats
        }
        # Add 'Unknown' category if needed
        if 'Unknown' not in color_map:
             color_map['Unknown'] = [128, 128, 128, default_color[3] if len(default_color) == 4 else 180] # Grey for unknowns

        df['__color_r'] = df[color_column].astype(str).map(lambda x: color_map.get(x, color_map['Unknown'])[0])
        df['__color_g'] = df[color_column].astype(str).map(lambda x: color_map.get(x, color_map['Unknown'])[1])
        df['__color_b'] = df[color_column].astype(str).map(lambda x: color_map.get(x, color_map['Unknown'])[2])
        df['__color_a'] = df[color_column].astype(str).map(lambda x: color_map.get(x, color_map['Unknown'])[3])

        return df, '[__color_r, __color_g, __color_b, __color_a]'


# --- Refactored Pydeck Functions ---

def create_pydeck_scatterplot(gdf: gpd.GeoDataFrame,
                              color_column: Optional[str] = None,
                              size_column: Optional[str] = None,
                              size_scale: float = 50.0,
                              tooltip_cols: Optional[List[str]] = None,
                              map_style: str = 'light',
                              initial_zoom: int = 3):
    """
    Creates an interactive scatterplot map using PyDeck, plotting centroids.

    Args:
        gdf: Input GeoDataFrame. Assumed to have a 'geometry' column.
        color_column: Column name to use for point color.
        size_column: Column name to use for point size (radius).
                     Uses square root scaling for better visual perception.
        size_scale: Scaling factor for point radius.
        tooltip_cols: List of column names to include in the tooltip.
                      Defaults to common useful columns if None.
        map_style: Pydeck map style (e.g., 'light', 'dark', 'satellite').
        initial_zoom: Initial zoom level for the map.

    Returns:
        pydeck.Deck: A PyDeck map object, or None if input is invalid.
    """
    if gdf is None or gdf.empty:
        print("Input GeoDataFrame is empty or None.")
        return None
    if 'geometry' not in gdf.columns:
        print("GeoDataFrame must have a 'geometry' column.")
        return None

    print(f"Creating Pydeck scatterplot for {len(gdf)} features...")

    # Ensure correct CRS
    if gdf.crs != "EPSG:4326":
        print(f"Reprojecting GDF from {gdf.crs} to EPSG:4326...")
        gdf = gdf.to_crs("EPSG:4326")

    # Prepare DataFrame for Pydeck
    df = pd.DataFrame()
    # Use centroid for plotting points
    gdf['__centroid'] = gdf.geometry.centroid
    df['lon'] = gdf['__centroid'].x
    df['lat'] = gdf['__centroid'].y
    gdf = gdf.drop(columns=['__centroid']) # Clean up temporary column

    # Copy relevant attribute columns
    potential_tooltip_cols = ['unified_id', 'dataset', 'area_m2', 'capacity_mw', 'install_date']
    if tooltip_cols is None:
        tooltip_cols = [col for col in potential_tooltip_cols if col in gdf.columns]
    else:
        # Ensure requested tooltip columns exist
        tooltip_cols = [col for col in tooltip_cols if col in gdf.columns]

    # Include color and size columns in the DataFrame if they exist and are needed
    cols_to_copy = set(tooltip_cols)
    if color_column and color_column in gdf.columns:
        cols_to_copy.add(color_column)
    if size_column and size_column in gdf.columns:
        cols_to_copy.add(size_column)

    for col in cols_to_copy:
        df[col] = gdf[col]

    # --- Color Mapping ---
    df, color_accessor = get_pydeck_color(df, color_column)

    # --- Size Mapping ---
    if size_column and size_column in gdf.columns and pd.api.types.is_numeric_dtype(df[size_column]):
        # Use sqrt scaling for better visual perception of area/capacity
        # Handle potential negative values if necessary before sqrt
        min_val = df[size_column].min()
        if min_val < 0:
             print(f"Warning: Negative values found in size column '{size_column}'. Clamping to 0 for sqrt.")
             df['__size_val'] = np.sqrt(df[size_column].clip(lower=0).fillna(0)) * size_scale
        else:
             df['__size_val'] = np.sqrt(df[size_column].fillna(0)) * size_scale
        size_accessor = '__size_val'
        print(f"Using column '{size_column}' for point size (sqrt scaled).")
    else:
        if size_column:
             print(f"Warning: Size column '{size_column}' not found or not numeric. Using fixed size.")
        size_accessor = size_scale # Fixed radius if no valid column
        print(f"Using fixed point size: {size_scale}")


    # --- Tooltip ---
    tooltip_html = ""
    for col in tooltip_cols:
        # Format column name nicely for display
        col_display = col.replace('_', ' ').title()
        tooltip_html += f"<b>{col_display}:</b> {{{col}}}<br>"
    tooltip = {"html": tooltip_html.strip('<br>')} if tooltip_html else None


    # --- Layer ---
    layer = pdk.Layer(
        'ScatterplotLayer',
        df,
        get_position=['lon', 'lat'],
        get_radius=size_accessor,
        get_fill_color=color_accessor,
        pickable=True,
        opacity=0.7,
        stroked=True,
        filled=True,
        radius_min_pixels=1,
        radius_max_pixels=100,
        get_line_color=[0, 0, 0, 100], # Faint black outline
        line_width_min_pixels=0.5
    )

    # --- View State ---
    view_state = pdk.ViewState(
        longitude=df['lon'].mean(),
        latitude=df['lat'].mean(),
        zoom=initial_zoom,
        pitch=0,
        bearing=0
    )

    # --- Deck ---
    deck = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip=tooltip,
        map_style=map_style
    )
    print("Pydeck scatterplot created successfully.")
    return deck


def create_pydeck_polygons(gdf: gpd.GeoDataFrame,
                           color_column: Optional[str] = None,
                           extrusion_column: Optional[str] = None,
                           extrusion_scale: float = 1.0,
                           tooltip_cols: Optional[List[str]] = None,
                           map_style: str = 'light',
                           initial_zoom: int = 10,
                           sample_frac: float = 1.0, # Add sampling parameter
                           where_clause: Optional[str] = None): # Add filtering parameter
    """
    Creates an interactive 3D polygon map using PyDeck.
    NOTE: Rendering large numbers of complex polygons can be slow.
          Consider using sample_frac or where_clause for large datasets.

    Args:
        gdf: Input GeoDataFrame. Must contain Polygon/MultiPolygon geometries.
        color_column: Column name to use for polygon color.
        extrusion_column: Column name to use for polygon height extrusion.
                          Uses linear scaling.
        extrusion_scale: Scaling factor for extrusion height. Applied AFTER normalization if numeric.
        tooltip_cols: List of column names to include in the tooltip.
        map_style: Pydeck map style.
        initial_zoom: Initial zoom level.
        sample_frac: Fraction of data to sample (0.0 to 1.0). 1.0 means no sampling.
        where_clause: A Pandas query string to filter the GDF before visualization.

    Returns:
        pydeck.Deck: A PyDeck map object, or None if input is invalid.
    """
    if gdf is None or gdf.empty:
        print("Input GeoDataFrame is empty or None.")
        return None
    if 'geometry' not in gdf.columns:
        print("GeoDataFrame must have a 'geometry' column.")
        return None

    # --- Filtering and Sampling ---
    original_count = len(gdf)
    if where_clause:
        try:
            gdf = gdf.query(where_clause).copy()
            print(f"Filtered GDF using '{where_clause}'. Count: {len(gdf)} (from {original_count})")
        except Exception as e:
            print(f"Error applying where_clause '{where_clause}': {e}. Using original GDF.")
            gdf = gdf.copy() # Ensure we have a copy
    else:
         gdf = gdf.copy() # Ensure we have a copy

    if sample_frac < 1.0 and len(gdf) > 0:
        try:
            gdf = gdf.sample(frac=sample_frac, random_state=42) # Use random_state for reproducibility
            print(f"Sampled {sample_frac*100:.1f}% of data. Count: {len(gdf)} (from {original_count if not where_clause else 'filtered'})")
        except Exception as e:
             print(f"Error sampling data: {e}. Using full (or filtered) GDF.")
    # --- End Filtering and Sampling ---


    # Ensure correct CRS
    if gdf.crs != "EPSG:4326":
        print(f"Reprojecting GDF from {gdf.crs} to EPSG:4326...")
        gdf = gdf.to_crs("EPSG:4326")

    # Filter for valid Polygon/MultiPolygon geometries
    gdf = gdf[gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon']) & gdf.geometry.is_valid & ~gdf.geometry.is_empty]

    if gdf.empty:
        print("No valid Polygon or MultiPolygon geometries found after filtering.")
        return None

    print(f"Creating Pydeck polygon map for {len(gdf)} features...")

    # Prepare data for Pydeck PolygonLayer
    # Using __geo_interface__ is generally more efficient than manual iteration
    gdf['__geojson_feature'] = gdf.apply(lambda row: row.geometry.__geo_interface__, axis=1)

    # Extract properties into the DataFrame for easier access by Pydeck accessors
    potential_tooltip_cols = ['unified_id', 'dataset', 'area_m2', 'capacity_mw', 'install_date']
    if tooltip_cols is None:
        tooltip_cols = [col for col in potential_tooltip_cols if col in gdf.columns]
    else:
        tooltip_cols = [col for col in tooltip_cols if col in gdf.columns] # Ensure they exist

    cols_to_keep = set(tooltip_cols)
    if color_column and color_column in gdf.columns:
        cols_to_keep.add(color_column)
    if extrusion_column and extrusion_column in gdf.columns:
        cols_to_keep.add(extrusion_column)
    cols_to_keep.add('__geojson_feature') # Add the geometry representation

    df_pydeck = gdf[list(cols_to_keep)].copy()


    # --- Color Mapping ---
    df_pydeck, color_accessor = get_pydeck_color(df_pydeck, color_column, default_color=[0, 128, 0, 150]) # Slightly transparent default

    # --- Extrusion Mapping ---
    if extrusion_column and extrusion_column in gdf.columns and pd.api.types.is_numeric_dtype(df_pydeck[extrusion_column]):
        # Normalize extrusion height for better scaling, handle NaNs
        min_val = df_pydeck[extrusion_column].min()
        max_val = df_pydeck[extrusion_column].max()
        if pd.isna(min_val) or pd.isna(max_val) or max_val == min_val:
             print(f"Warning: Cannot normalize extrusion column '{extrusion_column}'. Using fixed extrusion.")
             df_pydeck['__elevation_val'] = 1.0 # Use a base value
        else:
             df_pydeck['__elevation_val'] = (df_pydeck[extrusion_column].fillna(min_val) - min_val) / (max_val - min_val)

        # Apply overall scale factor
        elevation_accessor = f'__elevation_val * {extrusion_scale * 1000}' # Multiply scale for visibility
        print(f"Using column '{extrusion_column}' for extrusion (normalized & scaled by {extrusion_scale*1000}).")
        extruded = True
    else:
        if extrusion_column:
            print(f"Warning: Extrusion column '{extrusion_column}' not found or not numeric. No extrusion applied.")
        elevation_accessor = 0 # No height
        extruded = False
        print("No extrusion applied.")


    # --- Tooltip ---
    tooltip_html = ""
    for col in tooltip_cols:
        col_display = col.replace('_', ' ').title()
        tooltip_html += f"<b>{col_display}:</b> {{{col}}}<br>"
    tooltip = {"html": tooltip_html.strip('<br>')} if tooltip_html else None

    # --- Layer ---
    layer = pdk.Layer(
        'GeoJsonLayer', # Use GeoJsonLayer which handles Polygon/MultiPolygon via __geo_interface__
        df_pydeck,
        opacity=0.7,
        stroked=True, # Wireframe
        filled=True,
        extruded=extruded,
        wireframe=True,
        get_polygon='__geojson_feature', # Access the geojson geometry
        get_fill_color=color_accessor,
        get_line_color=[0, 0, 0, 100],
        get_line_width=10, # Meters
        line_width_min_pixels=0.5,
        get_elevation=elevation_accessor if extruded else 0,
        pickable=True,
        auto_highlight=True
    )

    # --- View State ---
    # Calculate bounds more efficiently
    bounds = gdf.total_bounds # [minx, miny, maxx, maxy]
    center_lon = (bounds[0] + bounds[2]) / 2
    center_lat = (bounds[1] + bounds[3]) / 2

    view_state = pdk.ViewState(
        longitude=center_lon,
        latitude=center_lat,
        zoom=initial_zoom,
        pitch=45 if extruded else 0, # Pitch only if extruded
        bearing=0
    )

    # --- Deck ---
    deck = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip=tooltip,
        map_style=map_style
    )
    print("Pydeck polygon map created successfully.")
    return deck


def create_pydeck_heatmap(gdf: gpd.GeoDataFrame,
                          weight_column: Optional[str] = None,
                          radius_pixels: int = 50,
                          intensity: float = 1.0,
                          threshold: float = 0.05,
                          aggregation: str = 'SUM', # SUM or MEAN
                          tooltip_cols: Optional[List[str]] = None, # Tooltip not directly supported by HeatmapLayer
                          map_style: str = 'light',
                          initial_zoom: int = 3):
    """
    Creates an interactive heatmap using PyDeck.

    Args:
        gdf: Input GeoDataFrame.
        weight_column: Column name to use for heatmap weighting. If None, uses count.
        radius_pixels: Radius of influence for each point in pixels.
        intensity: Multiplier factor for heatmap intensity.
        threshold: Minimum threshold for color rendering (0 to 1).
        aggregation: 'SUM' or 'MEAN' - how weights are accumulated.
        tooltip_cols: (Currently ignored by HeatmapLayer) List of columns for tooltip.
        map_style: Pydeck map style.
        initial_zoom: Initial zoom level.

    Returns:
        pydeck.Deck: A PyDeck map object, or None if input is invalid.
    """
    if gdf is None or gdf.empty:
        print("Input GeoDataFrame is empty or None.")
        return None
    if 'geometry' not in gdf.columns:
        print("GeoDataFrame must have a 'geometry' column.")
        return None

    print(f"Creating Pydeck heatmap for {len(gdf)} features...")

    # Ensure correct CRS
    if gdf.crs != "EPSG:4326":
        print(f"Reprojecting GDF from {gdf.crs} to EPSG:4326...")
        gdf = gdf.to_crs("EPSG:4326")

    # Prepare DataFrame for Pydeck
    df = pd.DataFrame()
    gdf['__centroid'] = gdf.geometry.centroid
    df['lon'] = gdf['__centroid'].x
    df['lat'] = gdf['__centroid'].y

    # --- Weight Mapping ---
    if weight_column and weight_column in gdf.columns and pd.api.types.is_numeric_dtype(gdf[weight_column]):
         # Handle NaNs - fill with 0 or mean/median depending on desired behavior
        if gdf[weight_column].isnull().any():
            print(f"Warning: NaN values found in weight column '{weight_column}'. Filling with 0.")
            df['__weight'] = gdf[weight_column].fillna(0)
        else:
            df['__weight'] = gdf[weight_column]
        weight_accessor = '__weight'
        print(f"Using column '{weight_column}' for heatmap weights (Aggregation: {aggregation}).")
    else:
        if weight_column:
            print(f"Warning: Weight column '{weight_column}' not found or not numeric. Using count (weight=1).")
        weight_accessor = 1 # Use count if no valid weight column
        print(f"Using count (weight=1) for heatmap.")

    # --- Layer ---
    # Viridis color range (adjust alpha as needed)
    color_range = [
        [68, 1, 84, 255],
        [72, 40, 120, 255],
        [62, 74, 137, 255],
        [49, 104, 142, 255],
        [38, 130, 142, 255],
        [31, 158, 137, 255],
        [53, 183, 121, 255],
        [109, 205, 89, 255],
        [180, 222, 44, 255],
        [253, 231, 37, 255]
    ]

    layer = pdk.Layer(
        'HeatmapLayer',
        df,
        get_position=['lon', 'lat'],
        get_weight=weight_accessor,
        opacity=0.8,
        radius_pixels=radius_pixels,
        intensity=intensity,
        threshold=threshold,
        aggregation=aggregation.upper(), # Ensure uppercase
        color_range=color_range # Use Viridis colormap
    )

    # --- View State ---
    view_state = pdk.ViewState(
        longitude=df['lon'].mean(),
        latitude=df['lat'].mean(),
        zoom=initial_zoom,
        pitch=0,
        bearing=0
    )

    # --- Deck ---
    # Tooltip is generally not useful for HeatmapLayer
    deck = pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        map_style=map_style,
        tooltip=False # Disable tooltip for heatmap
    )
    print("Pydeck heatmap created successfully.")
    return deck


# STAC Collections and fetching samples with cubo

[cubo](https://cubo.readthedocs.io/en/latest/index.html) is a Python library for working with STAC (SpatioTemporal Asset Catalog) collections and fetching samples from them. It provides a convenient way to interact with STAC APIs and retrieve geospatial data.

Look into managing raster data with duckdb or similar:
- https://www.linkedin.com/pulse/querying-stac-load-satellite-imagery-sentinel2-duckdb-alvaro-huarte-yjuzf/?trackingId=wfMPNnd%2BDh2zi1AoiLG2vg%3D%3D


In [ ]:
import cubo 
import xarray as xr
import rioxarray as rxr

def percentile_normalize(da, lower_percentile=5, upper_percentile=95):
    """Normalize using percentiles to improve visualization contrast"""
    # Calculate percentiles per band
    mins = da.quantile(lower_percentile/100, dim=('x', 'y'))
    maxs = da.quantile(upper_percentile/100, dim=('x', 'y'))
    
    # Apply normalization
    normalized = (da - mins) / (maxs - mins)
    return normalized.clip(0, 1)

# use cubo to fetch a series of 4 tiles and visualize per their tutorial: https://cubo.readthedocs.io/en/latest/tutorials/getting_started.html
def fetch_cubo_stac_rasters_samples(
    pv_gdf: gpd.GeoDataFrame,
    stac_url='https://planetarycomputer.microsoft.com/api/stac/v1',
    collection='sentinel-2-l2a',
    bands=['B02', 'B03', 'B04'],
    start_date='2023-01-01',
    end_date='2023-03-31',
    units="m",
    edge_size=1280,
    resolution=10,
    sample_size=1,
    visualize_set=False,
):
    """
    Fetch raster samples from a STAC collection using Cubo and vi
    Args:
        pv_gdf (GeoDataFrame): Input GeoDataFrame with geometry column.
        stac_url (str): STAC API URL.
        collection (str): STAC collection name.
        start_date (str): Start date for filtering.
        end_date (str): End date for filtering.
        units (str): Units for the raster data.
        edge_size (int): Size of the edges of the raster tiles.
        resolution (int): Resolution of the raster data.
        sample_size (int): Number of samples to fetch.
    Returns:
        stac_xr_samples (xr.DataArray): Sampled raster data.
"""

    # Sort by area descending and sample
    sampled_gdf = pv_gdf.sort_values(by='area_m2', ascending=False)
    sampled_gdf = sampled_gdf[:1000].sample(sample_size, random_state=42)
    # all should be using 'EPSG:4326'
    target_epsg = sampled_gdf.crs
    default_query = {"eo:cloud_cover": {"lt": 25}}
    
    stac_items = []
    print(f"Fetching samples for {len(sampled_gdf)} locations...")
    for idx, row in sampled_gdf.iterrows():
        pv_lat, pv_long = row.geometry.centroid.y, row.geometry.centroid.x
        print(f"Fetching samples for {idx}: {pv_lat}, {pv_long} with area {row['area_m2']:.2f} m2")
        # Fetch samples using Cubo
        try:
            # Fetch samples using Cubo
            # Pass the EPSG code as an integer, not a string with "EPSG:" prefix
            pv_da = cubo.create(
                lat=pv_lat,
                lon=pv_long,
                stac=stac_url,
                collection=collection,
                bands=bands,
                start_date=start_date,
                end_date=end_date,
                edge_size=1280,
                units=units,
                resolution=resolution,
                # Use integer EPSG code instead of string with 'EPSG:' prefix
                # stackstac_kw=dict( # stackstac keyword arguments
                #     # xy_coords='center',
                #     epsg=32611)
                query=default_query
            )

            # return pv_da
            
            stac_items.append(pv_da)
                
        except Exception as e:
            print(f"Error fetching or visualizing data for {idx}: {e}")
    

    if not stac_items:
        print("No valid data items fetched.")
        return None


    # visualize only last sample for now
    if visualize_set and pv_da is not None:
        # Check that the 'time' coordinate exists and has data
        if "time" in pv_da.coords and len(pv_da.time) > 0:
            # Calculate number of columns for the visualization grid
            axes = min(4, len(pv_da.time))
            # Display RGB composite by selecting red, green, blue bands and scaling
            pv_da = pv_da.groupby('time').first()
            rgb_da = percentile_normalize(pv_da.sel(band=["B04", "B03", "B02"]))
            rgb_da.plot.imshow(col="time", col_wrap=axes)
        
        else:
            print(f"No data available for location {pv_lat}, {pv_long}")
    
    try:
        # First ensure all datasets have same dimensions and variables
        aligned_items = []
        for item in stac_items:
            # Ensure all datasets use the same CRS
            if item.rio.crs is None:
                item = item.rio.write_crs("EPSG:4326")
            
            # Add to aligned list
            aligned_items.append(item)
        
        # Use rioxarray's merge functionality which handles spatial contexts better
        # First combine by time (similar to your original approach)
        merged_data = xr.concat(aligned_items, dim='time')
        
        # For spatial merging with overlapping tiles, you could use:
        # merged_data = rxr.merge.merge_arrays(aligned_items)
        
        return merged_data
        
    except Exception as e:
        print(f"Error merging datasets: {e}")
        # If merge fails, return the list of items instead
        print("Returning list of individual DataArrays without merging")
        return stac_items



In [ ]:
stac_items = fetch_cubo_stac_rasters_samples(ds_gdf, visualize_set=True, sample_size=3)
stac_items

In [ ]:
# visualize one of the samples
np.random.seed(42)
# print(pv_da)
# pv_da = samples.isel(time=0)
pv_uniq = stac_items.groupby('time').first()
(pv_uniq.sel(band=["B04","B03","B02"])/5000).clip(0,1).plot.imshow(col="time",col_wrap = 3)

In [ ]:
def create_temporal_animation(stacked_data, output_path="pv_animation.gif"):
    """Create an animated GIF showing temporal changes"""
    import matplotlib.pyplot as plt
    import matplotlib.animation as animation
    
    # Prepare RGB data
    rgb_data = []
    for t in stacked_data.time.values:
        rgb = visualize_enhanced_rgb(stacked_data.sel(time=t))
        rgb_data.append(rgb.transpose("y", "x", "band").values)
    
    # Create animation
    fig, ax = plt.figure(figsize=(10, 10)), plt.gca()
    
    # Create initial image
    im = ax.imshow(rgb_data[0])
    ax.set_axis_off()
    
    # Define update function for animation
    def update(frame):
        im.set_array(rgb_data[frame])
        ax.set_title(f"Date: {stacked_data.time.values[frame]}")
        return [im]
    
    # Create and save animation
    ani = animation.FuncAnimation(fig, update, frames=len(rgb_data), interval=500)
    ani.save(output_path, writer='pillow')
    
    return ani

In [ ]:
crerate_temporal_animation(stac_items, output_path="pv_animation.gif")

In [ ]:
# sample a single centroid pv polygon from the gdf


In [ ]:
# import pystac_client
# import pystac
# import geopandas as gpd
# from shapely.geometry import box
# import matplotlib.pyplot as plt
# import warnings

# # Suppress specific UserWarning from pystac_client about ItemSearch link
# warnings.filterwarnings("ignore", message="Server does not conform to ITEM_SEARCH.*", category=UserWarning)

# # --- Configuration ---
# MAXAR_OPEN_DATA_CATALOG_URL = "https://maxar-opendata.s3.amazonaws.com/events/catalog.json"

# # --- STAC Traversal ---
# print(f"Connecting to catalog: {MAXAR_OPEN_DATA_CATALOG_URL}...")
# all_items = []
# try:
#     # Open the root STAC Catalog
#     root_catalog = pystac_client.Client.open(MAXAR_OPEN_DATA_CATALOG_URL)
#     print("Successfully connected to root catalog.")

#     # Get top-level children (likely Collections or Catalogs representing events)
#     print("Fetching top-level children (Events)...")
#     top_children = list(root_catalog.get_children())
#     print(f"Found {len(top_children)} top-level children (Events).")

#     # Iterate through the top-level children (Events)
#     for event_child in top_children:
#         print(f"\nProcessing Event Child: {event_child.id} ({type(event_child)})...")
#         try:
#             # Attempt to get items directly from this event child
#             # get_items() should work for both Catalog and Collection objects if they contain items
#             print(f"  Fetching items from {event_child.id}...")
#             # Resolve the child first if it's just a link (get_children often returns resolved)
#             # event_child.resolve_stac_object() # Might be needed if get_children returns unresolved links

#             items_in_event = list(event_child.get_items(recursive=False)) # Use recursive=True just in case
#             if items_in_event:
#                  print(f"    Found {len(items_in_event)} items directly in {event_child.id}.")
#                  all_items.extend(items_in_event)
#           #   else:
#           #        # If no items directly, check if it has children itself (deeper nesting)
#           #        print(f"    No items found directly in {event_child.id}. Checking for sub-children...")
#           #        try:
#           #             sub_children = list(event_child.get_children())
#           #             if sub_children:
#           #                  print(f"    Found {len(sub_children)} sub-children in {event_child.id}.")
#           #                  for sub_child in sub_children:
#           #                       print(f"      Fetching items from sub-child: {sub_child.id}...")
#           #                       try:
#           #                            items_in_sub = list(sub_child.get_items(recursive=True))
#           #                            print(f"        Found {len(items_in_sub)} items in {sub_child.id}.")
#           #                            all_items.extend(items_in_sub)
#           #                       except Exception as sub_item_err:
#           #                            print(f"        Error fetching items from {sub_child.id}: {sub_item_err}")
#           #             else:
#           #                  print(f"    No sub-children found in {event_child.id}.")
#           #        except Exception as sub_child_err:
#           #             print(f"    Error fetching sub-children for {event_child.id}: {sub_child_err}")

#         except AttributeError as ae:
#              # Handle cases where an object might not have get_items or get_children
#              print(f"  Skipping {event_child.id}, does not appear to be a Catalog/Collection or error occurred: {ae}")
#         except Exception as event_err:
#             print(f"  Error processing {event_child.id}: {event_err}")


#     print(f"\nTotal items fetched from all events: {len(all_items)}")

#     # --- Data Extraction and Preparation --- (Same as before)
#     if not all_items:
#         print("No items found after traversing. Exiting.")
#     else:
#         print("Extracting bounding boxes...")
#         bboxes = []
#         item_ids = []
#         for item in all_items:
#             if item.bbox:
#                 bboxes.append(item.bbox)
#                 item_ids.append(item.id)
#             else:
#                 # It's also possible geometry exists but not bbox
#                 if item.geometry:
#                      try:
#                           # Calculate bbox from geometry (more expensive)
#                           min_lon, min_lat, max_lon, max_lat = gpd.GeoSeries([item.geometry], crs="EPSG:4326").total_bounds
#                           bboxes.append([min_lon, min_lat, max_lon, max_lat])
#                           item_ids.append(item.id)
#                           print(f"Info: Item {item.id} using geometry bounds.")
#                      except Exception as geom_err:
#                           print(f"Warning: Item {item.id} has no bbox and failed to get bounds from geometry: {geom_err}")
#                 else:
#                      print(f"Warning: Item {item.id} has no bbox or geometry.")


#         if not bboxes:
#              print("No items with bounding boxes found. Cannot plot.")
#         else:
#             # Convert bounding boxes (min_lon, min_lat, max_lon, max_lat) to Shapely polygons
#             geometries = [box(min_lon, min_lat, max_lon, max_lat) for min_lon, min_lat, max_lon, max_lat in bboxes]

#             # Create a GeoDataFrame
#             gdf_items = gpd.GeoDataFrame({'id': item_ids, 'geometry': geometries}, crs="EPSG:4326")
#             print(f"Created GeoDataFrame with {len(gdf_items)} item extents.")
#             print(gdf_items.head()) # Display first few rows

#             # --- Plotting --- (Same as before)
#             print("Plotting item extents...")
#             world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#             fig, ax = plt.subplots(1, 1, figsize=(15, 10))
#             world.plot(ax=ax, color='lightgray', edgecolor='black')
#             gdf_items.plot(ax=ax, color='red', alpha=0.5, edgecolor='red', linewidth=0.5) # Adjusted linewidth
#             ax.set_title(f'Maxar Open Data Catalog Item Extents ({len(gdf_items)} items)')
#             ax.set_xlabel('Longitude')
#             ax.set_ylabel('Latitude')
#             # Consider setting limits based on data extent if it's geographically concentrated
#             minx, miny, maxx, maxy = gdf_items.total_bounds
#             ax.set_xlim(minx - 2, maxx + 2) # Add padding
#             ax.set_ylim(miny - 2, maxy + 2)
#             plt.tight_layout() # Adjust layout
#             plt.show()
#             print("Plotting complete.")

# except Exception as e:
#     print(f"An error occurred during catalog processing: {e}")

In [ ]:
# from maxar_platform.session import session
# session.login()

In [ ]:
# from maxar_platform.discovery import open_catalog
# mxr_catalog = open_catalog('imagery') # returns the root catalog, or pass a subcatalog name like "imagery"
# results = mxr_catalog.search(
#     max_items=10,
#     # rough Puerto Rico Bounding Box
#     bbox=[-67.3507516333,17.5877264027,-65.532680798,19.0597280009],
#     # from 2017-2024
#     datetime=['2017-01-01T00:00:00Z', '2024-12-31T23:59:59Z']
# )

In [ ]:
# open_catalog('imagery')

In [ ]:
# for item in results.items():
#     print(item.id)

In [ ]:
# import rasterio.plot
# test_item = next(results.items_as_dicts())
# print(f"Processing item: {test_item['id']}")
# print(f"Item keys: {test_item.keys()}")
# print(f"Item assets: {test_item['assets']['browse'].keys()}")
# with rasterio.open(test_item['assets']["browse"]['href']) as dataset:
#     rasterio.plot.show(dataset)

In [ ]:
# Args:
#         gdf: Input GeoDataFrame. Assumed to have a 'geometry' column.
#         color_column: Column name to use for point color.
#         size_column: Column name to use for point size (radius).
#                      Uses square root scaling for better visual perception.
#         size_scale: Scaling factor for point radius.
#         tooltip_cols: List of column names to include in the tooltip.
#                       Defaults to common useful columns if None.
#         map_style: Pydeck map style (e.g., 'light', 'dark', 'satellite').
#         initial_zoom: Initial zoom level for the map.
create_pydeck_scatterplot(ds_gdf, color_column='dataset', map_style='light', tooltip_cols=['dataset', 'area_m2'])

In [ ]:
# heatmap_scale = 1000
# gdf: gpd.GeoDataFrame,
# color_column: Optional[str] = None,
# extrusion_column: Optional[str] = None,
# extrusion_scale: float = 1.0,
# tooltip_cols: Optional[List[str]] = None,
# map_style: str = 'light',
# initial_zoom: int = 10,
# sample_frac: float = 1.0, # Add sampling parameter
# where_clause: Optional[str] = None): # Add filtering parameter
# create_pydeck_polygons(
#     gdf=ds_gdf,
#     color_column='dataset',
#     extrusion_column='area_m2',
#     extrusion_scale=1000000.0,
#     tooltip_cols=['dataset', 'area_m2'],
#     map_style='light',
#     initial_zoom=3,
#     sample_frac=0.25, # 10% of data
#     where_clause=None
# )

In [ ]:
# see pydeck map styles here: 
create_pydeck_heatmap(
    gdf=ds_gdf,
    weight_column='area_m2',
    radius_pixels=20,
    intensity=3.,
    threshold=0.00005,
    aggregation='MEAN', # SUM or MEAN
    tooltip_cols=['dataset', 'area_m2'],
    map_style='light',
    initial_zoom=4
)

In [ ]:
# filter out NaN area
# ds_gdf = ds_gdf[ds_gdf['area_m2'].notna()]
# # create_pydeck_polygons(ds_gdf, color_column='area_m2', extrusion_column='area_m2', title=f"{ds} - Capacity 3D Map")
# create_pydeck_polygons(ds_gdf, extrusion_column='area_m2', title=f"{ds} - PV installations area 3D Map")

## Lonboard Visualization Functions

Lonboard is a GPU-accelerated geospatial visualization library that's excellent for handling very large datasets. It's particularly useful for creating high-performance interactive visualizations of millions of data points.

In [ ]:
def create_lonboard_map(gdf, color_column=None, size_column=None, size_scale=1,
                       title="PV Installation Map"):
    """
    Create an interactive map of PV installations using Lonboard.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame containing PV installation data with geometry column
    color_column : str, optional
        Column name to use for point coloring
    size_column : str, optional
        Column name to use for point sizing
    size_scale : float
        Scaling factor for point size
    title : str
        Title for the map
        
    Returns:
    --------
    lonboard.Map
        Interactive Lonboard map
    """
    # Ensure the GeoDataFrame is in WGS84 (EPSG:4326)
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Handle color mapping if specified
    if color_column and color_column in gdf.columns:
        color = gdf[color_column]
    else:
        color = None
    
    # Handle size mapping if specified
    if size_column and size_column in gdf.columns:
        size = gdf[size_column] * size_scale
    else:
        size = size_scale
    
    # Create the map
    m = lonboard.Map()
    
    # Handle different geometry types
    if all(gdf.geometry.geom_type.isin(['Point'])):
        # For point geometries
        m.add_layer(
            lonboard.ScatterplotLayer(
                gdf,
                get_color=color,
                get_radius=size,
                opacity=0.8,
                pickable=True,
                auto_highlight=True
            )
        )
    elif all(gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])):
        # For polygon geometries
        m.add_layer(
            lonboard.GeoJsonLayer(
                gdf,
                get_fill_color=color,
                get_line_color=[0, 0, 0, 200],
                get_line_width=2,
                opacity=0.8,
                pickable=True,
                auto_highlight=True
            )
        )
    else:
        # For mixed geometries, convert to points (centroids) for simplicity
        gdf_centroids = gdf.copy()
        gdf_centroids['geometry'] = gdf_centroids.geometry.centroid
        
        m.add_layer(
            lonboard.ScatterplotLayer(
                gdf_centroids,
                get_color=color,
                get_radius=size,
                opacity=0.8,
                pickable=True,
                auto_highlight=True
            )
        )
    
    return m

def create_lonboard_heatmap(gdf, weight_column=None, radius=1000,
                          intensity=1, title="PV Installation Heatmap"):
    """
    Create a heatmap of PV installations using Lonboard.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame containing PV installation data with geometry column
    weight_column : str, optional
        Column name to use for heatmap weighting
    radius : float
        Radius of influence for each point (in meters)
    intensity : float
        Intensity of the heatmap
    title : str
        Title for the map
        
    Returns:
    --------
    lonboard.Map
        Interactive Lonboard heatmap
    """
    # Ensure the GeoDataFrame is in WGS84 (EPSG:4326)
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Handle weight mapping if specified
    if weight_column and weight_column in gdf.columns:
        weight = gdf[weight_column]
    else:
        weight = 1
    
    # Get centroids for all geometries
    gdf_centroids = gdf.copy()
    if not all(gdf.geometry.geom_type.isin(['Point'])):
        gdf_centroids['geometry'] = gdf_centroids.geometry.centroid
    
    # Create the map
    m = lonboard.Map()
    
    # Add heatmap layer
    m.add_layer(
        lonboard.HeatmapLayer(
            gdf_centroids,
            get_weight=weight,
            radius_pixels=int(radius/100),  # Convert meters to pixels roughly
            intensity=intensity,
            threshold=0.05,
            color_range=[
                [1, 152, 189, 255],
                [73, 227, 206, 255],
                [216, 254, 181, 255],
                [254, 237, 177, 255],
                [254, 173, 84, 255],
                [209, 55, 78, 255]
            ]
        )
    )
    
    return m

def create_lonboard_aggregation(gdf, resolution=8, color_scale='viridis',
                              title="PV Installation Density"):
    """
    Create a hexbin aggregation map of PV installations using Lonboard.
    
    Parameters:
    -----------
    gdf : GeoDataFrame
        GeoDataFrame containing PV installation data with geometry column
    resolution : int
        Resolution of hexbins (higher = more detailed)
    color_scale : str
        Matplotlib colormap name for coloring
    title : str
        Title for the map
        
    Returns:
    --------
    lonboard.Map
        Interactive Lonboard hexbin aggregation map
    """
    # Ensure the GeoDataFrame is in WGS84 (EPSG:4326)
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Get centroids for all geometries
    gdf_centroids = gdf.copy()
    if not all(gdf.geometry.geom_type.isin(['Point'])):
        gdf_centroids['geometry'] = gdf_centroids.geometry.centroid
    
    # Create the map
    m = lonboard.Map()
    
    # Add hexbin layer
    m.add_layer(
        lonboard.H3HexagonLayer(
            gdf_centroids,
            get_hex_id=lambda row: h3.geo_to_h3(row.geometry.y, row.geometry.x, resolution),
            get_fill_color="colorScale",
            color_scale=color_scale,
            opacity=0.8,
            pickable=True,
            auto_highlight=True
        )
    )
    
    return m

In [ ]:
# create_lonboard_aggregation(ds_gdf, resolution=7, color_scale='viridis', title=f"{ds} - Density Map")
# create_lonboard_map(ds_gdf, color_column='area_m2', size_column='area_sqm', size_scale=100, title=f"{ds} - Capacity Map")
# create_lonboard_heatmap(ds_gdf, weight_column='capacity_mw', radius=1000, intensity=1, title=f"{ds} - Capacity Heatmap")

<!-- from init generated visualization code with copilot -->
## Notes

These visualization functions provide a comprehensive toolkit for exploring and presenting your PV installation data. Each library has its strengths:

- **Folium**: Best for quick interactive web maps with various basemaps and standard visualization types
- **PyDeck**: Excellent for 3D visualizations and handling larger datasets with complex visualizations
- **Lonboard**: Best performance for very large datasets with GPU acceleration